In [1]:
!wget "https://github.com/freemedom/temp_dataset/releases/download/v2.0/pokemon_stats_20.json"

--2025-11-14 22:40:22--  https://github.com/freemedom/temp_dataset/releases/download/v2.0/pokemon_stats_20.json
Resolving github.com (github.com)... failed: Temporary failure in name resolution.
wget: unable to resolve host address ‘github.com’


In [2]:
# Pokemon Battles Prediction 2025 - Enhanced Version
# FDS Kaggle Competition Solution - Advanced Feature Engineering

"""
===============================================================================
                          特征工程总结
===============================================================================

本方案使用了约233个精心设计的特征，分为三大类：

【一、静态特征 (60个)】- 来自队伍和Pokemon基础属性
    
    1. P1队伍属性聚合 (30个)
       - 6个基础属性: HP, ATK, DEF, SPA, SPD, SPE
       - 5种聚合方式: sum(总和), mean(均值), max(最大), min(最小), std(标准差)
       - 特征示例: p1_team_hp_sum, p1_team_atk_mean, p1_team_spe_max
       
    2. P2首发Pokemon属性 (6个)
       - p2_lead_hp, p2_lead_atk, p2_lead_def
       - p2_lead_spa, p2_lead_spd, p2_lead_spe
       
    3. 属性对比特征 (12个)
       - 优势值 (6个): hp_advantage, atk_advantage, def_advantage 等
       - 比率 (6个): hp_ratio, atk_ratio, def_ratio 等
       
    4. 类型克制特征 (4个)
       - type_adv_mean: 队伍对P2首发的平均类型优势
       - type_adv_max: 最大类型优势
       - type_adv_min: 最小类型优势  
       - type_adv_std: 类型优势标准差
       
    5. 队伍多样性特征 (2个)
       - type_diversity: 类型多样性（唯一类型数/总类型数）
       - stat_diversity: 属性多样性（变异系数）
       
    6. 队伍平衡性特征 (3个)
       - physical_special_atk_ratio: 物攻/特攻比
       - physical_special_def_ratio: 物防/特防比
       - offense_defense_ratio: 攻击/防御比
       
    7. 总体实力特征 (3个)
       - p1_total_stats: P1队伍总属性值
       - p2_total_stats: P2首发总属性值
       - total_stats_advantage: 总属性优势

【二、动态特征 (174个)】- 来自战斗时间线分析

    1. 基础信息 (1个)
       - total_turns: 战斗总回合数
       
    2. 整体HP统计 (14个)
       - 每个玩家7个: start(起始), end(结束), min(最低), max(最高),
                      avg(平均), std(标准差), trend(趋势线斜率)
       - 特征示例: p1_hp_start, p2_hp_end, p1_hp_trend
       
    3. 分段HP统计 (12个) ⭐新增
       - 三个阶段: early(前1/3), mid(中1/3), late(后1/3)
       - 每个阶段每个玩家2个: avg(平均HP), min(最低HP)
       - 特征示例: p1_early_hp_avg, p2_late_hp_min
       
    4. HP损失速率 (4个)
       - p1_avg_hp_loss, p1_max_hp_loss: P1平均和最大HP损失
       - p2_avg_hp_loss, p2_max_hp_loss: P2平均和最大HP损失
       
    5. HP优势指标 (3个)
       - hp_advantage_start: 起始HP优势
       - hp_advantage_end: 结束HP优势  
       - hp_advantage_avg: 平均HP优势
       
    6. 招式多样性 (2个)
       - p1_move_diversity: P1招式多样性（唯一招式数/总招式数）
       - p2_move_diversity: P2招式多样性
       
    7. 状态异常总计 (2个)
       - p1_status_changes: P1状态异常回合数
       - p2_status_changes: P2状态异常回合数
       
    8. 异常状态统计 (1个) ⭐新增
       - abnormal_status_count_ratio: P1前30回合出现的精灵中有异常状态的精灵数量 / (P2前30回合出现的精灵中有异常状态的精灵数量 + 1)
       - 说明：统计前30回合出现的所有精灵（最多6只，也可能不足6只）中，最终状态为异常状态（非nostatus，非fnt）的精灵数量比值
       - 使用平滑处理避免除以0，比值反映双方受异常状态影响的相对程度
       
    9. Counter技能统计 (2个) ⭐新增
       - p1_counter_invalid: P1使用Counter但无效的次数
       - p2_counter_invalid: P2使用Counter但无效的次数
       - 说明：Counter只能反弹物理攻击，对手使用特殊/状态技或换人时无效
       
   10. 招式威力统计 (6个)
       - 每个玩家3个: avg_move_power, max_move_power, min_move_power
       - 特征示例: p1_avg_move_power, p2_max_move_power
       
   11. 招式命中率 (2个)
       - p1_avg_accuracy: P1平均命中率
       - p2_avg_accuracy: P2平均命中率
       
   12. 招式类别比例 (6个)
       - 每个玩家3个: physical_move_ratio, special_move_ratio, status_move_ratio
       - 分别统计物理、特殊、变化招式的使用比例
       
   13. 换精灵统计 (10个) ⭐增强
       - p1_switch_count, p2_switch_count: 换精灵次数
       - p1_early_switch_ratio, p2_early_switch_ratio: 早期换精灵比例
       - p1_move_null_count, p2_move_null_count: move_details为null的总次数（保留用于兼容）
       - p1_move_null_switch, p2_move_null_switch: 因换人导致的move_details为null的次数 ⭐新增
       - p1_move_null_status, p2_move_null_status: 因状态异常导致的move_details为null的次数 ⭐新增
       
   14. 技能使用次数统计 (80个) ⭐新增
       - 每个技能的使用次数: P1和P2各40个技能
       - 特征格式: p1_move_{skill_name}_count, p2_move_{skill_name}_count
       - 统计战斗过程中每个技能被使用的总次数
       - 例如: p1_move_thunderbolt_count, p2_move_ice_beam_count
       
   15. 连续攻击统计 (2个) ⭐新增
       - p1_max_consecutive_attacks: P1最大连续攻击回合数
       - p2_max_consecutive_attacks: P2最大连续攻击回合数
       
   16. 属性提升 (Boost) 统计 (13个)
       - 每个玩家6个: final_boost_sum, final_atk_boost, final_def_boost,
                      final_spa_boost, final_spd_boost, final_spe_boost
       - boost_advantage: P1和P2的boost总和差值
       
   17. 场地效果统计 (2个)
       - p1_effect_turns: P1受效果影响的回合数
       - p2_effect_turns: P2受效果影响的回合数
       
   18. 前30回合精灵个数统计 (2个) ⭐新增
       - p1_unique_pokemon_count_30turns: P1在前30回合内出现的不同精灵个数
       - p2_unique_pokemon_count_30turns: P2在前30回合内出现的不同精灵个数
       - 统计前30回合内每个玩家使用了多少种不同的精灵
       
   19. 前30回合6只精灵总HP统计 (3个) ⭐新增
       - p1_total_pokemon_hp_pct_30turns: P1的6只精灵在前30回合内的hp_pct总和
       - p2_total_pokemon_hp_pct_30turns: P2的6只精灵在前30回合内的hp_pct总和
       - total_pokemon_hp_pct_ratio_30turns: P1和P2前30回合总HP的比值 (p1/p2) ⭐新增
       - 对于每只出现的精灵，取它最后一次出现的hp_pct
       - 如果前30回合内出现的精灵不足6只，其余精灵按满血1.0计算
       - 最终总和 = 出现精灵的最后hp_pct之和 + (6 - 出现数量) × 1.0
       - 反映队伍整体在前30回合的血量状态和相对优势
       
   20. 死亡精灵数量统计 (2个) ⭐新增
       - fnt_count_ratio: P1死亡精灵数量 / (P2死亡精灵数量 + 1)
       - fnt_count_diff: P1死亡精灵数量 - P2死亡精灵数量
       - 统计战斗时间线中所有出现过的精灵，状态为'fnt'的数量
       - 比值特征反映双方的战损比（使用平滑处理避免除以0）
       - 差值特征反映绝对优势（正数表示P1损失更大，负数表示P2损失更大）

   21. 类型克制效果倍数 (2个) ⭐新增
       - p1_avg: 玩家1队伍对玩家2队伍的平均类型效果倍数
       - p2_avg: 玩家2队伍对玩家1队伍的平均类型效果倍数
       - 基于战斗过程中使用的技能类型，计算类型克制效果
       - 考虑技能威力和类型克制关系（超有效2倍、效果不佳0.5倍、无效0倍）
       - 反映双方队伍在类型上的相对优势
       
   22. 先制技能优先级总和 (2个) ⭐新增
       - p1_num_priority_moves: 玩家1先制技能的优先级总和
       - p2_num_priority_moves: 玩家2先制技能的优先级总和
       - 统计所有先制技能的优先级值总和，用于评估队伍的先制攻击能力
       - 优先级值越高，行动顺序越靠前，在战斗中具有速度优势

【三、交互特征 (4个)】- 组合特征

    1. hp_boost_interaction: HP优势 × Boost优势
       - 捕捉血量和属性提升的协同效应
       
    2. p1_effective_power: P1平均威力 × P1平均命中率
       - P1的有效攻击力（考虑命中）
       
    3. p2_effective_power: P2平均威力 × P2平均命中率
       - P2的有效攻击力（考虑命中）
       
    4. type_stats_interaction: 类型优势均值 × 总属性优势
       - 类型克制和属性优势的组合效应

【总计】
    - 静态特征: 60个
    - 动态特征: 179个 (原175个 + 2个类型克制效果倍数特征 + 2个先制技能优先级总和特征)  # 已注释掉第30回合组合胜率特征
    - 交互特征: 4个
    - 总特征数: 约243个

【特征工程亮点】
    ✨ 多维度聚合: 使用5种统计量全面描述队伍属性
    ✨ 时间分段分析: 将战斗分为早中晚期，捕捉不同阶段特征
    ✨ 完整类型克制: 基于Gen 1-9的完整类型克制表
    ✨ 战术特征: 连续攻击、换精灵时机等高级战术指标
    ✨ 特征交互: 捕捉多个特征之间的非线性关系

【模型集成策略】
    - 基础模型: RandomForest, XGBoost, LightGBM, CatBoost, GradientBoosting
    - 集成方法: Stacking (元学习器: LogisticRegression) + Voting (Soft)
    - 验证策略: 80/20分割 + 分层采样

【预期性能】
    - 验证集准确率: 82-86%
    - 相比基础方案提升: +2-4%
    
===============================================================================
"""

import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, RFE, RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import permutation_importance
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
import warnings
import os
import sys
import optuna
from optuna.pruners import MedianPruner


def extract_final_pokemon_status(timeline):
    """
    根据战斗时间线提取双方精灵的最终状态。
    
    返回:
    - p1_alive_pokemon_names: P1最终存活精灵名称集合
    - p1_fnt_pokemon_names: P1最终死亡精灵名称集合
    - p2_alive_pokemon_names: P2最终存活精灵名称集合
    """
    p1_alive_pokemon_names = set()
    p1_fnt_pokemon_names = set()
    p2_alive_pokemon_names = set()
    p1_last_status = {}
    p2_last_status = {}
    for turn in timeline:
        p1_pokemon_state = turn.get('p1_pokemon_state', {})
        p1_pokemon_name = p1_pokemon_state.get('name', '')
        if p1_pokemon_name:
            p1_last_status[p1_pokemon_name] = p1_pokemon_state.get('status', 'nostatus')

        p2_pokemon_state = turn.get('p2_pokemon_state', {})
        p2_pokemon_name = p2_pokemon_state.get('name', '')
        if p2_pokemon_name:
            p2_last_status[p2_pokemon_name] = p2_pokemon_state.get('status', 'nostatus')

    for pokemon_name, status in p1_last_status.items():
        if status == 'fnt':
            p1_fnt_pokemon_names.add(pokemon_name)
        else:
            p1_alive_pokemon_names.add(pokemon_name)

    for pokemon_name, status in p2_last_status.items():
        if status != 'fnt':
            p2_alive_pokemon_names.add(pokemon_name)

    return p1_alive_pokemon_names, p1_fnt_pokemon_names, p2_alive_pokemon_names

def extract_moves(data: dict):
    """
    提取战斗过程中每个玩家使用的技能集合。
    
    该函数遍历战斗时间线，记录每个宝可梦使用的所有技能。如果宝可梦被击败（fainted），
    则移除该宝可梦的技能记录，因为被击败的宝可梦不应该被计入最终的特征计算。
    
    Args:
        data (dict): 战斗数据字典，必须包含 "battle_timeline" 键，其中包含每个回合的信息
    
    Returns:
        p1_pokemon_moves (dict): 玩家1的宝可梦使用的技能字典
            格式: {宝可梦名: [技能详情1, 技能详情2, ...]}
        p2_pokemon_moves (dict): 玩家2的宝可梦使用的技能字典
            格式: {宝可梦名: [技能详情1, 技能详情2, ...]}
    """
    # 初始化两个字典，用于存储每个玩家每个宝可梦使用的技能列表
    p1_pokemon_moves = {}  # 玩家1的宝可梦技能字典
    p2_pokemon_moves = {}  # 玩家2的宝可梦技能字典

    # 遍历战斗时间线的每个回合
    for turn in data.get("battle_timeline", []):
        # 获取当前回合中两个玩家出场的宝可梦名称
        p1_name = turn.get("p1_pokemon_state", {}).get("name", "")  # 玩家1当前宝可梦名称
        p2_name = turn.get("p2_pokemon_state", {}).get("name", "")  # 玩家2当前宝可梦名称

        # 处理玩家1的技能记录
        # 条件：技能详情不为空 且 (该宝可梦还未记录 或 该技能还未被记录过)
        p1_move = turn.get("p1_move_details")
        if p1_move is not None and (p1_name not in p1_pokemon_moves or p1_move not in p1_pokemon_moves[p1_name]):
            # 如果该宝可梦不在字典中，创建空列表；否则使用现有列表
            # 然后添加新的技能详情（去重：相同技能不会重复添加）
            p1_pokemon_moves.setdefault(p1_name, []).append(p1_move)
        
        # 处理玩家2的技能记录（逻辑同上）
        p2_move = turn.get("p2_move_details")
        if p2_move is not None and (p2_name not in p2_pokemon_moves or p2_move not in p2_pokemon_moves[p2_name]):
            p2_pokemon_moves.setdefault(p2_name, []).append(p2_move)

        # 如果玩家1的宝可梦被击败（status == "fnt"），移除该宝可梦的技能记录
        # 因为被击败的宝可梦不应该参与后续的特征计算
        p1_status = turn.get("p1_pokemon_state", {}).get("status", "")
        if p1_status == "fnt":
            p1_pokemon_moves.pop(p1_name, None)  # 安全删除，如果键不存在也不报错
        
        # 如果玩家2的宝可梦被击败，移除该宝可梦的技能记录
        p2_status = turn.get("p2_pokemon_state", {}).get("status", "")
        if p2_status == "fnt":
            p2_pokemon_moves.pop(p2_name, None)  # 安全删除，如果键不存在也不报错

    return p1_pokemon_moves, p2_pokemon_moves

def type_multiplier(p1_moves: dict, p2_moves: dict):
    """
    计算每个队伍的平均类型效果倍数（type effectiveness multiplier）。
    
    该函数通过分析两个玩家使用的技能类型，计算每个玩家队伍对对方队伍的平均类型克制效果。
    类型克制关系：超有效（2倍）> 普通（1倍）> 效果不佳（0.5倍）> 无效（0倍）
    
    Args:
        p1_moves (dict): 玩家1的宝可梦使用的技能字典，格式为 {宝可梦名: [技能列表]}
        p2_moves (dict): 玩家2的宝可梦使用的技能字典，格式为 {宝可梦名: [技能列表]}
    
    Returns:
        p1_team_avg (float): 玩家1队伍对玩家2队伍的平均类型效果倍数
        p2_team_avg (float): 玩家2队伍对玩家1队伍的平均类型效果倍数
    """
    # 第一步：提取每个宝可梦使用的攻击技能类型和威力
    # 只保留有类型、有基础威力且不是状态类技能的攻击技能
    type_pokemon1 = {}  # 存储玩家1每个宝可梦的技能类型和威力信息
    type_pokemon2 = {}  # 存储玩家2每个宝可梦的技能类型和威力信息

    # 处理玩家1的宝可梦技能
    for pokemon, moves in p1_moves.items():
        type_pokemon1[pokemon] = []
        for move in moves:
            # 过滤条件：技能存在、有类型、有基础威力、不是状态类技能
            if move and move.get("type") and move.get("base_power") and move.get("category") != "STATUS":
                # 存储技能类型（首字母大写）和有效威力（基础威力 × 命中率）
                type_pokemon1[pokemon].append({
                    "type": move["type"].capitalize(),  # 技能类型，如 "Fire", "Water"
                    "power": move["base_power"] * move.get("accuracy", 1.0)  # 有效威力 = 基础威力 × 命中率
                })

    # 处理玩家2的宝可梦技能（逻辑同上）
    for pokemon, moves in p2_moves.items():
        type_pokemon2[pokemon] = []
        for move in moves:
            if move and move.get("type") and move.get("base_power") and move.get("category") != "STATUS":
                type_pokemon2[pokemon].append({
                    "type": move["type"].capitalize(),
                    "power": move["base_power"] * move.get("accuracy", 1.0)
                })

    # 第二步：计算类型克制效果倍数
    diz_multiplier_my_pokemon = {}      # 玩家1的每个宝可梦对玩家2队伍的平均效果倍数
    diz_multiplier_other_pokemon = {}   # 玩家2的每个宝可梦对玩家1队伍的平均效果倍数

    # 计算玩家1的宝可梦对玩家2队伍的克制效果
    for pokemon1, moves1 in type_pokemon1.items():
        total_effectiveness = []  # 存储该宝可梦对玩家2每个宝可梦的效果倍数

        # 遍历玩家2的每个宝可梦，计算克制效果
        for pokemon2, moves2 in type_pokemon2.items():
            multiplier = 1.0  # 初始倍数设为1.0（普通效果）

            # 遍历玩家1该宝可梦的每个技能
            for move in moves1:
                t_att = move["type"]  # 攻击技能的类型
                base_power = move["power"]  # 技能的有效威力
                # 从类型克制表中获取该攻击类型的效果列表
                if t_att not in TABLE_TYPE:
                    continue
                super_eff, meno_eff, no_eff = TABLE_TYPE[t_att]
                # super_eff: 超有效的防御类型列表（2倍伤害）
                # meno_eff: 效果不佳的防御类型列表（0.5倍伤害）
                # no_eff: 无效的防御类型列表（0倍伤害）

                # 遍历玩家2宝可梦的每个防御类型（宝可梦可能有多个类型）
                for t_def in P_DEF_TYPE.get(pokemon2.lower(), []):
                    # 根据类型克制关系调整倍数
                    if t_def in no_eff:
                        multiplier *= 0.0  # 无效：伤害为0
                    elif t_def in super_eff:
                        multiplier *= 2.0  # 超有效：伤害×2
                    elif t_def in meno_eff:
                        multiplier *= 0.5  # 效果不佳：伤害×0.5

                # 将技能威力归一化（除以100）并乘以类型倍数
                multiplier *= (base_power / 100.0)

            # 记录该宝可梦对玩家2该宝可梦的总效果倍数
            total_effectiveness.append(multiplier)

        # 计算该宝可梦对玩家2所有宝可梦的平均效果倍数
        if total_effectiveness:
            diz_multiplier_my_pokemon[pokemon1] = np.mean(total_effectiveness)
        else:
            diz_multiplier_my_pokemon[pokemon1] = 0.0

    # 计算玩家2的宝可梦对玩家1队伍的克制效果（逻辑同上，方向相反）
    for pokemon2, moves2 in type_pokemon2.items():
        total_effectiveness = []

        # 遍历玩家1的每个宝可梦
        for pokemon1, moves1 in type_pokemon1.items():
            multiplier = 1.0

            # 遍历玩家2该宝可梦的每个技能
            for move in moves2:
                t_att = move["type"]  # 攻击技能类型
                base_power = move["power"]  # 技能有效威力
                if t_att not in TABLE_TYPE:
                    continue
                super_eff, meno_eff, no_eff = TABLE_TYPE[t_att]

                # 遍历玩家1宝可梦的每个防御类型
                for t_def in P_DEF_TYPE.get(pokemon1.lower(), []):
                    # 根据类型克制关系调整倍数
                    if t_def in no_eff:
                        multiplier *= 0.0
                    elif t_def in super_eff:
                        multiplier *= 2.0
                    elif t_def in meno_eff:
                        multiplier *= 0.5

                # 将技能威力归一化并乘以类型倍数
                multiplier *= (base_power / 100.0)

            total_effectiveness.append(multiplier)

        # 计算该宝可梦对玩家1所有宝可梦的平均效果倍数
        if total_effectiveness:
            diz_multiplier_other_pokemon[pokemon2] = np.mean(total_effectiveness)
        else:
            diz_multiplier_other_pokemon[pokemon2] = 0.0

    # 第三步：计算整个队伍的平均类型效果倍数
    # 玩家1队伍的平均效果 = 玩家1所有宝可梦的平均效果倍数的平均值
    if diz_multiplier_my_pokemon:
        p1_team_avg = np.mean(list(diz_multiplier_my_pokemon.values()))
    else:
        p1_team_avg = 0.0
    # 玩家2队伍的平均效果 = 玩家2所有宝可梦的平均效果倍数的平均值
    if diz_multiplier_other_pokemon:
        p2_team_avg = np.mean(list(diz_multiplier_other_pokemon.values()))
    else:
        p2_team_avg = 0.0

    return p1_team_avg, p2_team_avg

def count_priority_moves(pokemon_moves: dict) -> int:
    """
    统计宝可梦使用的所有先制技能（priority moves）的优先级总和。
    
    先制技能可以在对手之前行动，优先级值越高，行动顺序越靠前。
    该函数计算所有先制技能的优先级值总和，用于评估队伍的先制攻击能力。
    
    Args:
        pokemon_moves (dict): 宝可梦技能字典，格式为 {宝可梦名: [技能详情1, 技能详情2, ...]}
            每个技能详情可能包含 "priority" 字段（优先级值）
    
    Returns:
        int: 所有先制技能的优先级总和，如果技能没有priority字段则视为0
    """
    # 生成器表达式：遍历所有宝可梦的所有技能，获取优先级值（默认为0）
    return sum(move.get("priority", 0) for moves in pokemon_moves.values() for move in moves)

# ============================================================================
# 类型克制关系表（用于type_multiplier特征计算）
# ============================================================================
# 宝可梦类型克制关系表
# 格式: {攻击类型: (超有效列表, 效果不佳列表, 无效列表)}
# - 超有效（super effective）: 伤害×2
# - 效果不佳（not very effective）: 伤害×0.5
# - 无效（no effect）: 伤害×0
TABLE_TYPE = {
    "Normal": ([], ["Rock", "Steel"], ["Ghost"]),
    "Fire": (["Grass", "Ice", "Bug", "Steel"],
             ["Fire", "Water", "Rock", "Dragon"], []),
    "Water": (["Fire", "Ground", "Rock"],
              ["Water", "Grass", "Dragon"], []),
    "Electric": (["Water", "Flying"],
                 ["Electric", "Grass", "Dragon"], ["Ground"]),
    "Grass": (["Water", "Ground", "Rock"],
              ["Fire", "Grass", "Poison", "Flying", "Bug", "Dragon", "Steel"], []),
    "Ice": (["Grass", "Ground", "Flying", "Dragon"],
            ["Fire", "Water", "Ice", "Steel"], []),
    "Fighting": (["Normal", "Ice", "Rock", "Dark", "Steel"],
                 ["Poison", "Flying", "Psychic", "Bug", "Fairy"], []),
    "Poison": (["Grass", "Fairy"],
               ["Poison", "Ground", "Rock", "Ghost"], []),
    "Ground": (["Fire", "Electric", "Poison", "Rock", "Steel"],
               ["Grass", "Bug"], ["Flying"]),
    "Flying": (["Grass", "Fighting", "Bug"],
               ["Electric", "Rock", "Steel"], []),
    "Psychic": (["Fighting", "Poison"],
                ["Psychic", "Steel"], ["Dark"]),
    "Bug": (["Grass", "Psychic", "Dark"],
            ["Fire", "Fighting", "Poison", "Flying", "Ghost", "Steel", "Fairy"], []),
    "Rock": (["Fire", "Ice", "Flying", "Bug"],
             ["Fighting", "Ground", "Steel"], []),
    "Ghost": (["Psychic", "Ghost"],
              ["Dark"], ["Normal"]),
    "Dragon": (["Dragon"],
               ["Steel"], ["Fairy"]),
    "Dark": (["Psychic", "Ghost"],
             ["Fighting", "Dark", "Fairy"], []),
    "Steel": (["Ice", "Rock", "Fairy"],
              ["Fire", "Water", "Electric", "Steel"], []),
    "Fairy": (["Fighting", "Dragon", "Dark"],
              ["Fire", "Poison", "Steel"], [])
}

# 宝可梦防御类型表（第一世代常用宝可梦）
# 格式: {宝可梦名称（小写）: [类型1, 类型2, ...]}
P_DEF_TYPE = {
    "starmie": ["psychic", "water"],
    "exeggutor": ["grass", "psychic"],
    "chansey": ["normal"],
    "snorlax": ["normal"],
    "tauros": ["normal"],
    "alakazam": ["psychic"],
    "jynx": ["ice", "psychic"],
    "slowbro": ["psychic", "water"],
    "gengar": ["ghost", "poison"],
    "rhydon": ["ground", "rock"],
    "zapdos": ["electric", "flying"],
    "cloyster": ["ice", "water"],
    "golem": ["ground", "rock"],
    "jolteon": ["electric"],
    "articuno": ["flying", "ice"],
    "persian": ["normal"],
    "lapras": ["ice", "water"],
    "dragonite": ["dragon", "flying"],
    "victreebel": ["grass", "poison"],
    "charizard": ["fire", "flying"]
}

# ============================================================================
# 日志系统：将print输出同时保存到文件
# ============================================================================
_log_file_handle = None
_original_print = print

def setup_logging(log_dir='print_log'):
    """
    设置日志系统，将print输出同时保存到文件
    
    参数:
    - log_dir: 日志文件保存目录（默认'print_log'）
    
    返回:
    - 日志文件路径
    """
    global _log_file_handle
    
    # 创建日志目录
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    
    # 生成带时间戳的日志文件名
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    log_file_path = os.path.join(log_dir, f'print_log_{timestamp}.log')
    
    # 打开日志文件（追加模式，UTF-8编码）
    _log_file_handle = open(log_file_path, 'w', encoding='utf-8')
    
    return log_file_path

def log_print(*args, **kwargs):
    """
    自定义print函数，同时输出到控制台和日志文件
    
    用法与标准print完全相同
    支持所有print参数：sep, end, file, flush等
    """
    global _log_file_handle, _original_print
    
    # 获取参数（如果指定了file参数，只输出到指定文件，不输出到日志）
    file_param = kwargs.get('file', None)
    
    # 输出到控制台（标准print）
    _original_print(*args, **kwargs)
    
    # 输出到日志文件（如果file参数未指定或为None，且日志文件已打开）
    if _log_file_handle is not None and file_param is None:
        try:
            # 获取sep和end参数
            sep = kwargs.get('sep', ' ')
            end = kwargs.get('end', '\n')
            
            # 将参数转换为字符串，使用指定的分隔符
            message = sep.join(str(arg) for arg in args) + end
            
            _log_file_handle.write(message)
            _log_file_handle.flush()  # 立即刷新到文件
        except Exception as e:
            # 如果写入文件失败，不影响控制台输出
            _original_print(f"警告: 写入日志文件失败: {e}", file=sys.stderr)

def close_logging():
    """关闭日志文件"""
    global _log_file_handle
    if _log_file_handle is not None:
        _log_file_handle.close()
        _log_file_handle = None

# 替换全局print函数
print = log_print

class PokemonBattlePredictorEnhanced:
    """Pokemon战斗预测器主类 - 增强版"""

    def __init__(self):
        self.train_data = None
        self.test_data = None
        self.features = None
        self.target = None
        self.models = {}
        self.feature_importance = {}
        self.scaler = StandardScaler()
        self.feature_selector = None
        self.X_val = None
        self.y_val = None
        self.best_model = None
        self.best_model_name = None
        self.all_moves_list = None  # 存储所有技能列表
        # self.turn30_winrates = None  # 存储第30回合宝可梦组合胜率数据

        # ========================================================================
        # 特征移除配置列表
        # ========================================================================
        # 在此列表中添加要移除的特征名称，这些特征将在特征合并后被移除
        # 示例：
        # self.features_to_remove = ['p1_hp_min', 'p2_hp_min']
        # self.features_to_remove = []  # 空列表表示不移除任何特征
        # ========================================================================
        # self.features_to_remove = ['p1_switch_count','p2_switch_count','p1_hp_min','p2_hp_min', 'p2_move_null_switch','p1_move_null_switch']  # 默认不移除任何特征 'p1_unique_pokemon_count_30turns','p2_unique_pokemon_count_30turns',
        self.features_to_remove = []

        # ========================================================================
        # 验证集分割配置
        # ========================================================================
        self.validation_split = 0.1  # 验证集比例（用于模型验证）

        # ========================================================================
        # Optuna超参数调优配置参数
        # ========================================================================
        # Optuna自动搜索最优超参数，使用MedianPruner提前终止低效试验
        # ========================================================================
        self.use_optuna_tuning = False  # 是否启用Optuna超参数调优（默认禁用）
        self.optuna_n_trials = 50  # Optuna搜索试验次数（每个模型独立搜索）
        self.optuna_timeout = 3600  # 单个模型优化超时时间（秒，默认1小时）
        self.optuna_cv_folds = 3  # 交叉验证折数（用于评估超参数性能）
        self.optuna_pruner_warmup = 5  # MedianPruner预热步数（前5个trial不剪枝）
        self.optuna_pruner_interval = 1  # MedianPruner检查间隔（每个fold检查一次）

        # Optuna相关状态变量
        self.optuna_best_params = {}  # 存储各模型的最优参数
        self.optuna_studies = {}  # 存储Optuna study对象

    # def get_all_moves_list(self, max_samples=10000):
    #     """
    #     从训练数据中获取所有技能列表
        
    #     参数:
    #     - max_samples: 最大采样数（用于快速获取技能列表）
    #     """
    #     if self.all_moves_list is not None:
    #         return self.all_moves_list
        
    #     print("\n正在收集所有技能列表...")
    #     all_moves = set()
        
    #     sample_size = min(max_samples, len(self.train_data))
    #     for idx in range(sample_size):
    #         row = self.train_data.iloc[idx]
    #         timeline = row.get('battle_timeline', [])
            
    #         for turn in timeline:
    #             p1_move = turn.get('p1_move_details')
    #             p2_move = turn.get('p2_move_details')
                
    #             if p1_move and p1_move.get('name'):
    #                 all_moves.add(p1_move.get('name'))
                
    #             if p2_move and p2_move.get('name'):
    #                 all_moves.add(p2_move.get('name'))
        
    #     self.all_moves_list = sorted(list(all_moves))
    #     print(f"✓ 找到 {len(self.all_moves_list)} 个唯一技能")
        
    #     return self.all_moves_list

    def load_data(self, train_path, test_path):
        """加载训练和测试数据"""
        print("正在加载数据...")

        # 加载训练数据
        train_records = []
        with open(train_path, 'r', encoding='utf-8') as f:
            for line in f:
                train_records.append(json.loads(line.strip()))

        # 加载测试数据
        test_records = []
        with open(test_path, 'r', encoding='utf-8') as f:
            for line in f:
                test_records.append(json.loads(line.strip()))

        self.train_data = pd.DataFrame(train_records)
        self.test_data = pd.DataFrame(test_records)

        print(f"训练数据加载完成: {len(self.train_data)} 条记录")
        print(f"测试数据加载完成: {len(self.test_data)} 条记录")

        return self.train_data, self.test_data

    # def load_turn30_winrates(self, winrate_path='pokemon_turn30_winrates.json'):
    #     """
    #     加载第30回合宝可梦组合胜率数据
    #     
    #     参数:
    #     - winrate_path: 胜率数据文件路径
    #     
    #     返回:
    #     - winrate_dict: 胜率字典，如果文件不存在则返回None
    #     """
    #     if not os.path.exists(winrate_path):
    #         print(f"⚠️ 胜率数据文件不存在: {winrate_path}")
    #         print("   提示: 请先运行 extract_pokemon_turn30_data.py 生成胜率数据")
    #         return None
    #     
    #     try:
    #         with open(winrate_path, 'r', encoding='utf-8') as f:
    #             self.turn30_winrates = json.load(f)
    #         print(f"✓ 已加载第30回合组合胜率数据: {len(self.turn30_winrates)} 个组合")
    #         return self.turn30_winrates
    #     except Exception as e:
    #         print(f"⚠️ 加载胜率数据失败: {e}")
    #         return None

    def get_complete_type_effectiveness(self):
        """完整的Pokemon类型克制关系表(Gen 1-9)"""
        effectiveness = {
            'normal': {'rock': 0.5, 'ghost': 0.0, 'steel': 0.5},
            'fire': {'fire': 0.5, 'water': 0.5, 'grass': 2.0, 'ice': 2.0, 'bug': 2.0, 'rock': 0.5, 'dragon': 0.5, 'steel': 2.0},
            'water': {'fire': 2.0, 'water': 0.5, 'grass': 0.5, 'ground': 2.0, 'rock': 2.0, 'dragon': 0.5},
            'electric': {'water': 2.0, 'electric': 0.5, 'grass': 0.5, 'ground': 0.0, 'flying': 2.0, 'dragon': 0.5},
            'grass': {'fire': 0.5, 'water': 2.0, 'grass': 0.5, 'poison': 0.5, 'ground': 2.0, 'flying': 0.5, 'bug': 0.5, 'rock': 2.0, 'dragon': 0.5, 'steel': 0.5},
            'ice': {'fire': 0.5, 'water': 0.5, 'grass': 2.0, 'ice': 0.5, 'ground': 2.0, 'flying': 2.0, 'dragon': 2.0, 'steel': 0.5},
            'fighting': {'normal': 2.0, 'ice': 2.0, 'poison': 0.5, 'flying': 0.5, 'psychic': 0.5, 'bug': 0.5, 'rock': 2.0, 'ghost': 0.0, 'dark': 2.0, 'steel': 2.0, 'fairy': 0.5},
            'poison': {'grass': 2.0, 'poison': 0.5, 'ground': 0.5, 'rock': 0.5, 'ghost': 0.5, 'steel': 0.0, 'fairy': 2.0},
            'ground': {'fire': 2.0, 'electric': 2.0, 'grass': 0.5, 'poison': 2.0, 'flying': 0.0, 'bug': 0.5, 'rock': 2.0, 'steel': 2.0},
            'flying': {'electric': 0.5, 'grass': 2.0, 'fighting': 2.0, 'bug': 2.0, 'rock': 0.5, 'steel': 0.5},
            'psychic': {'fighting': 2.0, 'poison': 2.0, 'psychic': 0.5, 'dark': 0.0, 'steel': 0.5},
            'bug': {'fire': 0.5, 'grass': 2.0, 'fighting': 0.5, 'poison': 0.5, 'flying': 0.5, 'psychic': 2.0, 'ghost': 0.5, 'dark': 2.0, 'steel': 0.5, 'fairy': 0.5},
            'rock': {'fire': 2.0, 'ice': 2.0, 'fighting': 0.5, 'ground': 0.5, 'flying': 2.0, 'bug': 2.0, 'steel': 0.5},
            'ghost': {'normal': 0.0, 'psychic': 2.0, 'ghost': 2.0, 'dark': 0.5},
            'dragon': {'dragon': 2.0, 'steel': 0.5, 'fairy': 0.0},
            'dark': {'fighting': 0.5, 'psychic': 2.0, 'ghost': 2.0, 'dark': 0.5, 'fairy': 0.5},
            'steel': {'fire': 0.5, 'water': 0.5, 'electric': 0.5, 'ice': 2.0, 'rock': 2.0, 'steel': 0.5, 'fairy': 2.0},
            'fairy': {'fire': 0.5, 'fighting': 2.0, 'poison': 0.5, 'dragon': 2.0, 'dark': 2.0, 'steel': 0.5}
        }
        return effectiveness

    def calculate_team_type_advantage(self, p1_team, p2_lead):
        """
        计算队伍整体类型优势
        
        功能：
        计算P1队伍中每个Pokemon对P2首发的类型克制优势，并返回统计值
        
        参数：
        - p1_team: P1队伍列表，每个元素是一个Pokemon字典
        - p2_lead: P2首发Pokemon字典
        
        返回：
        - type_adv_mean: 队伍平均类型优势（所有Pokemon类型优势的均值）
        - type_adv_max: 最大类型优势（队伍中最强的类型优势）
        - type_adv_min: 最小类型优势（队伍中最弱的类型优势）
        - type_adv_std: 类型优势标准差（反映队伍类型优势的离散程度）
        
        计算逻辑：
        1. 对于P1队伍中的每个Pokemon，计算其对P2首发的类型优势
        2. 类型优势 = 所有类型克制关系的乘积（例如：火对草2.0 × 草对水2.0 = 4.0）
        3. 如果Pokemon有多个类型，每个类型对P2每个类型的克制关系都会相乘
        4. 最终返回所有Pokemon类型优势的统计值（均值、最大值、最小值、标准差）
        
        示例：
        - P1有火系Pokemon，P2首发是草系 → 优势 = 2.0（火克制草）
        - P1有火+飞行系Pokemon，P2首发是草+虫系 → 优势 = 2.0 × 2.0 = 4.0（火克草2.0，飞行克虫2.0）
        """
        # 获取完整的类型克制关系表（Gen 1-9）
        effectiveness = self.get_complete_type_effectiveness()

        advantages = []
        # 获取P2首发的所有类型
        p2_types = p2_lead.get('types', []) if p2_lead else []

        # 遍历P1队伍中的每个Pokemon
        for pokemon in p1_team:
            if not pokemon:
                continue
            # 获取当前Pokemon的所有类型
            p1_types = pokemon.get('types', [])

            # 初始化类型优势为1.0（中性，无克制）
            adv = 1.0
            # 遍历P1 Pokemon的每个类型
            for p1_type in p1_types:
                if p1_type.lower() in effectiveness:
                    # 遍历P2首发的每个类型
                    for p2_type in p2_types:
                        # 如果P1类型对P2类型有克制关系，则相乘累积
                        # 例如：火对草 = 2.0，火对虫 = 2.0，则火+飞行对草+虫 = 2.0 × 2.0 = 4.0
                        if p2_type.lower() in effectiveness[p1_type.lower()]:
                            adv *= effectiveness[p1_type.lower()][p2_type.lower()]
            # 将当前Pokemon的类型优势添加到列表
            advantages.append(adv)

        # 如果没有有效的Pokemon，返回默认值（中性优势）
        if not advantages:
            return {'type_adv_mean': 1.0, 'type_adv_max': 1.0, 'type_adv_min': 1.0}

        # 返回类型优势的统计值
        return {
            'type_adv_mean': np.mean(advantages),  # 平均类型优势
            'type_adv_max': max(advantages),       # 最大类型优势（队伍中最强的）
            'type_adv_min': min(advantages),       # 最小类型优势（队伍中最弱的）
            'type_adv_std': np.std(advantages) if len(advantages) > 1 else 0  # 标准差（至少2个Pokemon才计算）
        }

    def calculate_alive_teams_type_advantage(self, p1_alive_team, p2_alive_team):
        """
        计算P1存活精灵对P2存活精灵的类型优势
        
        功能：
        计算P1存活精灵对P2存活精灵的整体类型克制优势，并返回统计值
        
        参数：
        - p1_alive_team: P1存活精灵列表，每个元素是一个Pokemon字典
        - p2_alive_team: P2存活精灵列表，每个元素是一个Pokemon字典
        
        返回：
        - type_adv_mean: P1对P2的平均类型优势（所有P2精灵类型优势的均值）
        - type_adv_max: 最大类型优势（P1对某只P2精灵的最佳克制情况）
        - type_adv_min: 最小类型优势（P1对某只P2精灵的最差克制情况）
        - type_adv_std: 类型优势标准差（反映P1对不同P2精灵的类型优势差异）
        
        计算逻辑：
        1. 对于每个P2存活精灵，计算P1存活精灵对它的类型优势
        2. 使用 calculate_team_type_advantage 计算P1队伍对单个P2精灵的优势
        3. 收集所有P2精灵的类型优势值，计算统计值
        
        示例：
        - P1有3只存活精灵，P2有2只存活精灵
        - 对P2第1只精灵：P1优势 = 2.0
        - 对P2第2只精灵：P1优势 = 0.5
        - 返回：mean=1.25, max=2.0, min=0.5, std=0.75
        """
        if not p1_alive_team or not p2_alive_team:
            # 如果没有存活精灵，返回默认值（中性优势）
            return {
                'type_adv_mean': 1.0,
                'type_adv_max': 1.0,
                'type_adv_min': 1.0,
                'type_adv_std': 0.0
            }
        
        # 对于每个P2存活精灵，计算P1存活精灵对它的类型优势
        all_advantages = []
        for p2_pokemon in p2_alive_team:
            # 计算P1存活精灵对当前P2存活精灵的类型优势
            type_adv = self.calculate_team_type_advantage(p1_alive_team, p2_pokemon)
            # 使用均值作为该P2精灵的类型优势
            all_advantages.append(type_adv.get('type_adv_mean', 1.0))
        
        # 计算所有P2存活精灵的类型优势统计值
        if all_advantages:
            return {
                'type_adv_mean': np.mean(all_advantages),  # 平均类型优势
                'type_adv_max': np.max(all_advantages),    # 最大类型优势
                'type_adv_min': np.min(all_advantages),    # 最小类型优势
                'type_adv_std': np.std(all_advantages) if len(all_advantages) > 1 else 0.0  # 标准差
            }
        else:
            # 如果没有有效的优势值，返回默认值
            return {
                'type_adv_mean': 1.0,
                'type_adv_max': 1.0,
                'type_adv_min': 1.0,
                'type_adv_std': 0.0
            }

    def calculate_team_diversity(self, team):
        """
        计算队伍多样性
        
        参数:
        - team: Pokemon对象列表，每个对象包含types和base属性
        
        返回:
        - dict: 包含两个多样性指标的字典
            - type_diversity: 类型多样性（0-1之间，值越大表示类型越多样）
            - stat_diversity: 属性多样性（变异系数的平均值，值越大表示属性分布越分散）
        
        计算逻辑:
        1. 类型多样性：
           - 收集队伍中所有Pokemon的类型（包括双类型）
           - 计算唯一类型数 / 总类型数
           - 例如：6只Pokemon，共12个类型，但只有8种唯一类型 → 8/12 = 0.67
           - 值越接近1.0表示类型越多样，越接近0表示类型重复越多
        
        2. 属性多样性：
           - 收集所有Pokemon的6个基础属性（hp, atk, def, spa, spd, spe）
           - 对每个属性计算变异系数（CV = std / mean）
           - 变异系数反映该属性在队伍中的分散程度
           - 将6个属性的变异系数求平均，得到整体属性多样性
           - 值越大表示队伍属性分布越不均匀（有高有低），值越小表示属性分布越均匀
        """
        if not team:
            return {'type_diversity': 0, 'stat_diversity': 0}

        # 类型多样性：收集所有类型，计算唯一类型占比
        all_types = []
        for pokemon in team:
            if pokemon and 'types' in pokemon:
                all_types.extend(pokemon['types'])  # 扩展类型列表（双类型Pokemon会有2个类型）
        # 唯一类型数 / 总类型数（包括重复）
        # 例如：[fire, fire, water, grass, fire, water] → 唯一类型3种，总数6 → 3/6 = 0.5
        type_diversity = len(set(all_types)) / max(len(all_types), 1)

        # 属性多样性：使用变异系数（Coefficient of Variation, CV）衡量
        # 收集所有Pokemon的6个基础属性
        stats_matrix = []
        for pokemon in team:
            if pokemon:
                stats = [
                    pokemon.get('base_hp', 0),
                    pokemon.get('base_atk', 0),
                    pokemon.get('base_def', 0),
                    pokemon.get('base_spa', 0),
                    pokemon.get('base_spd', 0),
                    pokemon.get('base_spe', 0)
                ]
                stats_matrix.append(stats)

        stat_diversity = 0
        if stats_matrix:
            stats_matrix = np.array(stats_matrix)
            # 对每个属性（列）计算变异系数：CV = std / mean
            # 变异系数反映该属性在队伍中的相对分散程度
            for i in range(6):
                col = stats_matrix[:, i]  # 第i个属性的所有Pokemon的值
                if np.mean(col) > 0:
                    # CV = 标准差 / 平均值，值越大表示该属性在队伍中差异越大
                    stat_diversity += np.std(col) / np.mean(col)
            # 将6个属性的变异系数求平均，得到整体属性多样性
            stat_diversity /= 6

        return {'type_diversity': type_diversity, 'stat_diversity': stat_diversity}

    def extract_static_features(self):
        """
        提取静态特征（队伍和Pokemon属性）- 增强版
        
        共60个特征：
        - P1队伍属性聚合: 30个 (6属性 × 5聚合方式)
        - P2首发属性: 6个
        - 属性对比: 12个 (6个advantage + 6个ratio)
        - 类型优势: 4个 (mean, max, min, std)
        - 队伍多样性: 2个 (type_diversity, stat_diversity)
        - 队伍平衡性: 3个 (物攻/特攻比, 物防/特防比, 攻防比)
        - 总体实力: 3个 (p1_total, p2_total, advantage)
        """
        print("\n=== 提取静态特征(增强版) ===")
        
        # 加载宝可梦属性数据库（用于查找精灵属性）
        pokemon_db = {}
        try:
            if os.path.exists('pokemon_stats_20.json'):
                with open('pokemon_stats_20.json', 'r', encoding='utf-8') as f:
                    pokemon_list = json.load(f)
                    for pokemon in pokemon_list:
                        pokemon_name = pokemon.get('name', '').lower()
                        if pokemon_name:
                            pokemon_db[pokemon_name] = pokemon
                print(f"✓ 已加载 {len(pokemon_db)} 种宝可梦属性数据")
        except Exception as e:
            print(f"⚠️ 加载宝可梦属性数据库失败: {e}")
            pokemon_db = {}

        def extract_pokemon_stats(pokemon):
            """提取单个Pokemon的属性"""
            if not pokemon:
                return [0] * 6

            return [
                pokemon.get('base_hp', 0),
                pokemon.get('base_atk', 0),
                pokemon.get('base_def', 0),
                pokemon.get('base_spa', 0),
                pokemon.get('base_spd', 0),
                pokemon.get('base_spe', 0)
            ]

        def calculate_team_stats(alive_pokemon_names=None):
            """
            计算队伍整体属性
            
            参数:
            - alive_pokemon_names: 存活的精灵名称集合
            
            注意：函数通过闭包访问pokemon_db，从数据库中查找Pokemon对象
            """
            if not alive_pokemon_names:
                return {'sum': [0]*6, 'mean': [0]*6, 'max': [0]*6, 'min': [0]*6, 'std': [0]*6}

            stats_matrix = []
            for pokemon_name in alive_pokemon_names:
                # 从pokemon_db中查找该精灵的属性
                pokemon_name_lower = pokemon_name.lower()
                if pokemon_name_lower in pokemon_db:
                    pokemon = pokemon_db[pokemon_name_lower]
                    stats = extract_pokemon_stats(pokemon)
                    stats_matrix.append(stats)
                # 如果数据库中找不到，跳过该精灵（属性为0）

            # 如果没有有效的pokemon，返回0值
            if not stats_matrix:
                return {'sum': [0]*6, 'mean': [0]*6, 'max': [0]*6, 'min': [0]*6, 'std': [0]*6}

            stats_matrix = np.array(stats_matrix)

            return {
                'sum': np.sum(stats_matrix, axis=0).tolist(),
                'mean': np.mean(stats_matrix, axis=0).tolist(),
                'max': np.max(stats_matrix, axis=0).tolist(),
                'min': np.min(stats_matrix, axis=0).tolist(),
                'std': np.std(stats_matrix, axis=0).tolist()
            }

        def extract_single_static_features(row):
            """
            提取单个样本的静态特征（统一函数，用于训练和测试数据）
            
            参数:
            - row: 单个样本的DataFrame行
            
            返回:
            - features: 包含所有静态特征的字典
            """
            features = {}

            # 从p1_team_details中获取所有P1精灵名称（初始为全部存活）
            p1_team = row.get('p1_team_details', [])
            
            
            timeline = row.get('battle_timeline', [])
            (
                p1_alive_pokemon_names,
                p1_fnt_pokemon_names,
                p2_alive_pokemon_names,
            ) = extract_final_pokemon_status(timeline)
            
            
            
            # 从p1_team中提取未死亡的精灵名称，然后与p1_alive_pokemon_names合并
            # 这样既包含未出场的存活精灵，也包含时间线中出现的存活精灵
            p1_team_names = {pokemon.get('name', '') for pokemon in p1_team 
                           if pokemon and pokemon.get('name', '') not in p1_fnt_pokemon_names}
            p1_alive_pokemon_names = p1_team_names | p1_alive_pokemon_names

            # Player 1 队伍特征（只计算存活的精灵）
            # 使用完整的p1_team和合并后的p1_alive_pokemon_names（包含所有存活的精灵，包括未出场的）
            p1_team_stats = calculate_team_stats(alive_pokemon_names=p1_alive_pokemon_names)

            # Player 2 队伍特征（只计算存活的精灵）
            # p2_team = row.get('p2_team_details', []) # 没有p2_team_details这个键
            p2_team_stats = calculate_team_stats(alive_pokemon_names=p2_alive_pokemon_names)

            # P1队伍属性聚合特征（30个特征：6个属性 × 5种聚合方式）
            # 生成特征：p1_team_hp_sum, p1_team_hp_mean, p1_team_hp_max, p1_team_hp_min, p1_team_hp_std 等
            # for agg_type in ['sum', 'mean', 'max', 'min', 'std']:
            #     for i, stat_name in enumerate(['hp', 'atk', 'def', 'spa', 'spd', 'spe']):
            #         features[f'p1_team_{stat_name}_{agg_type}'] = p1_team_stats[agg_type][i]

            # P2首发Pokemon基础属性特征（6个特征）
            # 生成特征：p2_lead_hp, p2_lead_atk, p2_lead_def, p2_lead_spa, p2_lead_spd, p2_lead_spe
            # for i, stat_name in enumerate(['hp', 'atk', 'def', 'spa', 'spd', 'spe']):
            #     features[f'p2_lead_{stat_name}'] = p2_stats[i]

            # 属性对比特征（12个特征：6个advantage + 6个ratio）
            # advantage: P1队伍平均属性值 - P2队伍平均属性值（差值特征）
            # ratio: P1队伍平均属性值 / (P2队伍平均属性值 + 1)（比值特征，+1避免除以0）
            for i, stat_name in enumerate(['hp', 'atk', 'def', 'spa', 'spd', 'spe']):
                features[f'{stat_name}_advantage'] = p1_team_stats['mean'][i] - p2_team_stats['mean'][i]
                features[f'{stat_name}_ratio'] = p1_team_stats['mean'][i] / (p2_team_stats['mean'][i] + 1)

            # 类型优势特征(增强版)
            # 使用P1存活精灵和P2存活精灵计算类型优势
            # 从pokemon_db中查找P1存活的精灵对象
            p1_alive_team = []
            for pokemon_name in p1_alive_pokemon_names:
                pokemon_name_lower = pokemon_name.lower()
                if pokemon_name_lower in pokemon_db:
                    p1_alive_team.append(pokemon_db[pokemon_name_lower])
            
            # 从pokemon_db中查找P2存活的精灵对象
            p2_alive_team = []
            for pokemon_name in p2_alive_pokemon_names:
                pokemon_name_lower = pokemon_name.lower()
                if pokemon_name_lower in pokemon_db:
                    p2_alive_team.append(pokemon_db[pokemon_name_lower])
            
            # 使用新函数计算P1存活精灵对P2存活精灵的类型优势
            type_adv = self.calculate_alive_teams_type_advantage(p1_alive_team, p2_alive_team)
            
            # 类型优势特征说明：
            # 这些特征反映P1存活精灵对P2存活精灵的整体类型克制关系
            # type_adv_mean: P1存活精灵对P2存活精灵的平均类型优势
            #   值越大表示P1整体类型优势越强，>1.0表示P1占优，<1.0表示P2占优
            #   例如：如果P1对P2的3只精灵优势分别为[2.0, 1.5, 0.5]，则mean = 1.33（P1略占优）
            features['type_adv_mean'] = type_adv['type_adv_mean']
            
            # type_adv_max: P1存活精灵对P2存活精灵的最大类型优势
            #   反映P1对P2某只精灵的最佳克制情况，值越大表示存在明显的类型克制
            #   例如：[2.0, 1.5, 0.5] → max = 2.0（P1对某只P2精灵有2倍优势）
            features['type_adv_max'] = type_adv['type_adv_max']
            
            # type_adv_min: P1存活精灵对P2存活精灵的最小类型优势
            #   反映P1对P2某只精灵的最差克制情况，值越小表示存在明显的被克制
            #   例如：[2.0, 1.5, 0.5] → min = 0.5（P1对某只P2精灵被克制，只有0.5倍）
            features['type_adv_min'] = type_adv['type_adv_min']
            
            # type_adv_std: P1存活精灵对P2存活精灵的类型优势标准差
            #   反映类型优势的离散程度，值越大表示P1对不同P2精灵的类型优势差异较大
            #   例如：[2.0, 1.5, 0.5] → std较大，说明优势分布不均匀
            #   例如：[1.2, 1.1, 1.0] → std较小，说明优势分布较均匀
            features['type_adv_std'] = type_adv['type_adv_std']

            # 队伍多样性
            # 从pokemon_db中查找P1存活的精灵对象
            p1_team_for_diversity = []
            for pokemon_name in p1_alive_pokemon_names:
                pokemon_name_lower = pokemon_name.lower()
                if pokemon_name_lower in pokemon_db:
                    p1_team_for_diversity.append(pokemon_db[pokemon_name_lower])
            p1_diversity = self.calculate_team_diversity(p1_team_for_diversity)
            features.update(p1_diversity)  # 保持原有特征名：type_diversity, stat_diversity
            
            # 从pokemon_db中查找P2存活的精灵对象
            p2_team_for_diversity = []
            for pokemon_name in p2_alive_pokemon_names:
                pokemon_name_lower = pokemon_name.lower()
                if pokemon_name_lower in pokemon_db:
                    p2_team_for_diversity.append(pokemon_db[pokemon_name_lower])
            p2_diversity = self.calculate_team_diversity(p2_team_for_diversity)
            features.update({f'p2_{k}': v for k, v in p2_diversity.items()})  # p2_type_diversity, p2_stat_diversity
            
            # P1和P2多样性比值
            p1_type_div = p1_diversity.get('type_diversity', 0)
            p2_type_div = p2_diversity.get('type_diversity', 0)
            p1_stat_div = p1_diversity.get('stat_diversity', 0)
            p2_stat_div = p2_diversity.get('stat_diversity', 0)
            
            # 比值：P1 / P2（+1e-6避免除以0）
            features['type_diversity_ratio'] = p1_type_div / (p2_type_div + 1e-6)
            features['stat_diversity_ratio'] = p1_stat_div / (p2_stat_div + 1e-6)

            # 队伍平衡性特征
            # P1 队伍平衡性
            p1_physical_atk = p1_team_stats['mean'][1]
            p1_special_atk = p1_team_stats['mean'][3]
            p1_physical_def = p1_team_stats['mean'][2]
            p1_special_def = p1_team_stats['mean'][4]

            features['physical_special_atk_ratio'] = p1_physical_atk / (p1_special_atk + 1)
            features['physical_special_def_ratio'] = p1_physical_def / (p1_special_def + 1)
            features['offense_defense_ratio'] = (p1_physical_atk + p1_special_atk) / (p1_physical_def + p1_special_def + 1)
            
            # P2 队伍平衡性
            p2_physical_atk = p2_team_stats['mean'][1]
            p2_special_atk = p2_team_stats['mean'][3]
            p2_physical_def = p2_team_stats['mean'][2]
            p2_special_def = p2_team_stats['mean'][4]

            features['p2_physical_special_atk_ratio'] = p2_physical_atk / (p2_special_atk + 1)
            features['p2_physical_special_def_ratio'] = p2_physical_def / (p2_special_def + 1)
            features['p2_offense_defense_ratio'] = (p2_physical_atk + p2_special_atk) / (p2_physical_def + p2_special_def + 1)
            
            # P1 和 P2 平衡性比值
            features['physical_special_atk_ratio_p1_p2'] = features['physical_special_atk_ratio'] / (features['p2_physical_special_atk_ratio'] + 1e-6)
            features['physical_special_def_ratio_p1_p2'] = features['physical_special_def_ratio'] / (features['p2_physical_special_def_ratio'] + 1e-6)
            features['offense_defense_ratio_p1_p2'] = features['offense_defense_ratio'] / (features['p2_offense_defense_ratio'] + 1e-6)

            # 总体实力指标
            # 使用平均值进行比较，因为P1和P2存活数量可能不同
            # 平均值可以公平地比较双方的实力水平
            # p1_team_stats['mean'] 是一个包含6个属性平均值的列表：[hp_mean, atk_mean, def_mean, spa_mean, spd_mean, spe_mean]
            # sum(p1_team_stats['mean']) 将这6个属性的平均值相加，得到P1队伍的平均总实力指标
            # 这样即使P1有6只存活、P2只有3只存活，也能公平比较双方的平均实力水平
            p1_total_stats_mean = sum(p1_team_stats['mean'])
            p2_total_stats_mean = sum(p2_team_stats['mean'])
            features['p1_total_stats_mean'] = p1_total_stats_mean
            features['p2_total_stats_mean'] = p2_total_stats_mean
            features['total_stats_advantage'] = p1_total_stats_mean - p2_total_stats_mean
            
            # 可选：如果仍需要总和特征（用于其他分析），但要注意存活数量差异
            # features['p1_total_stats_sum'] = sum(p1_team_stats['sum'])
            # features['p2_total_stats_sum'] = sum(p2_team_stats['sum'])
            # features['p1_alive_count'] = len(p1_alive_pokemon_names)
            # features['p2_alive_count'] = len(p2_alive_pokemon_names)

            return features

        # 为训练数据提取特征
        train_features = [extract_single_static_features(row) for _, row in self.train_data.iterrows()]

        # 为测试数据提取相同特征
        test_features = [extract_single_static_features(row) for _, row in self.test_data.iterrows()]

        self.train_static_features = pd.DataFrame(train_features)
        self.test_static_features = pd.DataFrame(test_features)

        print(f"静态特征提取完成: {self.train_static_features.shape[1]} 个特征")
        return self.train_static_features, self.test_static_features

    def extract_dynamic_features(self):
        """
        提取动态特征（战斗时间线）- 增强版
        
        共175个特征：
        - 回合数: 1个
        - 整体HP统计: 14个 (每个玩家7个: start/end/min/max/avg/std/trend)
        - 分段HP统计: 12个 (早中晚期 × 每个玩家2个)
        - HP损失速率: 4个 (每个玩家2个: avg/max)
        - HP优势: 3个 (start/end/avg)
        - 招式多样性: 2个
        - 状态异常总计: 2个
        - 异常状态统计: 1个 (abnormal_status_count_ratio) ⭐新增
          * P1前30回合出现的精灵中有异常状态的精灵数量 / (P2前30回合出现的精灵中有异常状态的精灵数量 + 1)
          * 使用平滑处理避免除以0
        - Counter技能统计: 2个 (Counter无效次数×2) ⭐新增
        - 招式威力: 6个 (每个玩家3个: avg/max/min)
        - 命中率: 2个
        - 招式类别比例: 6个 (每个玩家3个: physical/special/status)
        - 换精灵: 10个 (count×2 + early_ratio×2 + move_null_count×2 + move_null_switch×2 + move_null_status×2) ⭐增强
        - 技能使用次数: 80个 (每个技能×2玩家) ⭐新增
        - 连续攻击: 2个
        - 属性提升: 13个 (每个玩家6个 + advantage)
        - 场地效果: 2个
        - 前30回合精灵个数: 2个 (p1_unique_pokemon_count_30turns, p2_unique_pokemon_count_30turns) ⭐新增
        - 前30回合6只精灵总HP: 3个 (p1_total_pokemon_hp_pct_30turns, p2_total_pokemon_hp_pct_30turns, total_pokemon_hp_pct_ratio_30turns) ⭐新增
          * 对于出现的精灵，取最后一次出现的hp_pct
          * 不足6只时，其余按满血1.0计算
          * total_pokemon_hp_pct_ratio_30turns = p1_total / p2_total (比值特征)
        - 死亡精灵数量统计: 2个 (fnt_count_ratio, fnt_count_diff) ⭐新增
          * fnt_count_ratio: P1死亡数量 / (P2死亡数量 + 1) [平滑处理避免除以0]
          * fnt_count_diff: P1死亡数量 - P2死亡数量 [反映绝对优势]
        # - 第30回合组合胜率: 1个 (turn30_combination_winrate) ⭐新增
        #   * 基于第30回合时双方存活宝可梦组合的历史胜率
        #   * 完全匹配时使用历史胜率比值，否则使用默认值1.0（中性）
        #   * 反映该组合在历史数据中的表现
        - 类型克制效果倍数: 2个 (p1_avg, p2_avg) ⭐新增
          * p1_avg: 玩家1队伍对玩家2队伍的平均类型效果倍数
          * p2_avg: 玩家2队伍对玩家1队伍的平均类型效果倍数
          * 基于战斗过程中使用的技能类型，计算类型克制效果
          * 考虑技能威力和类型克制关系（超有效2倍、效果不佳0.5倍、无效0倍）
          * 反映双方队伍在类型上的相对优势
        - 先制技能优先级总和: 2个 (p1_num_priority_moves, p2_num_priority_moves) ⭐新增
          * p1_num_priority_moves: 玩家1先制技能的优先级总和
          * p2_num_priority_moves: 玩家2先制技能的优先级总和
          * 统计所有先制技能的优先级值总和，用于评估队伍的先制攻击能力
          * 优先级值越高，行动顺序越靠前，在战斗中具有速度优势
        """
        print("\n=== 提取动态特征(增强版) ===")
        
        # 加载第30回合组合胜率数据（如果存在）
        # if self.turn30_winrates is None:
        #     self.load_turn30_winrates()
        
        # 获取所有技能列表
        # all_moves_list = self.get_all_moves_list()
        
        # 加载宝可梦属性数据库（用于查找P2其他精灵的属性）
        pokemon_db = {}
        try:
            if os.path.exists('pokemon_stats_20.json'):
                with open('pokemon_stats_20.json', 'r', encoding='utf-8') as f:
                    pokemon_list = json.load(f)
                    for pokemon in pokemon_list:
                        pokemon_name = pokemon.get('name', '').lower()
                        if pokemon_name:
                            pokemon_db[pokemon_name] = pokemon
                print(f"✓ 已加载 {len(pokemon_db)} 种宝可梦属性数据")
        except Exception as e:
            print(f"⚠️ 加载宝可梦属性数据库失败: {e}")
            pokemon_db = {}
        
        # 创建技能名称到特征名的映射（处理特殊字符）
        def sanitize_move_name(move_name):
            """将技能名称转换为有效的特征名"""
            # 替换空格和特殊字符为下划线
            sanitized = move_name.replace(' ', '_').replace('-', '_').replace("'", '').replace('.', '')
            # 移除其他特殊字符
            sanitized = ''.join(c if c.isalnum() or c == '_' else '_' for c in sanitized)
            return sanitized.lower()
        
        # move_to_feature = {move: sanitize_move_name(move) for move in all_moves_list}

        # 定义强度值计算权重
        WEIGHT_HP = 1.0
        WEIGHT_ATTACK = 1.5  # 物理攻击和特殊攻击的平均权重
        WEIGHT_DEFENSE = 1.0  # 物理防御和特殊防御的平均权重
        WEIGHT_SPEED = 1.75  # 速度通常非常关键
        
        # 特殊宝可梦修正值
        SPECIAL_POKEMON_MODIFIERS = {
            "chansey": 1.2,
            "alakazam": 1.25,
            "snorlax": 1.15,
            "dragonite": 1.2,
            "zapdos": 1.1,
            "starmie": 1.05,
            "exeggutor": 1.05,
            "gengar": 1.1,
            "rhydon": 1.05,
            "cloyster": 1.05,
            "golem": 1.05,
            "jolteon": 1.05,
            "articuno": 1.05,
            "persian": 1.0,
            "lapras": 1.05,
            "charizard": 1.0,
            "victreebel": 1.0,
            "jynx": 1.0,
            "slowbro": 1.05,
            "tauros": 1.05
        }
        
        def calculate_pokemon_strength(pokemon_data):
            """
            计算Pokemon的强度值
            
            参数:
            - pokemon_data: Pokemon数据字典，包含base_hp, base_atk, base_def, base_spa, base_spd, base_spe
            
            返回:
            - 强度值（浮点数）
            """
            if not pokemon_data:
                return 0.0
            
            base_hp = pokemon_data.get('base_hp', 0)
            base_atk = pokemon_data.get('base_atk', 0)
            base_def = pokemon_data.get('base_def', 0)
            base_spa = pokemon_data.get('base_spa', 0)
            base_spd = pokemon_data.get('base_spd', 0)
            base_spe = pokemon_data.get('base_spe', 0)
            
            # 计算基础强度值
            hp_score = base_hp * WEIGHT_HP
            atk_score = base_atk * WEIGHT_ATTACK
            def_score = base_def * WEIGHT_DEFENSE
            spa_score = base_spa * WEIGHT_ATTACK  # 特攻使用与物攻相同的权重
            spd_score = base_spd * WEIGHT_DEFENSE  # 特防使用与物防相同的权重
            spe_score = base_spe * WEIGHT_SPEED
            
            strength = hp_score + atk_score + def_score + spa_score + spd_score + spe_score
            
            # 应用特殊宝可梦修正值
            pokemon_name = pokemon_data.get('name', '').lower()
            if pokemon_name in SPECIAL_POKEMON_MODIFIERS:
                strength *= SPECIAL_POKEMON_MODIFIERS[pokemon_name]
            
            return strength

        def analyze_battle_timeline(row, timeline, pokemon_db=None, turn30_winrates=None):
            """分析战斗时间线 - 包含早中晚期分段分析"""
            if not timeline:
                return {}
            
            if pokemon_db is None:
                pokemon_db = {}
            
            if turn30_winrates is None:
                turn30_winrates = {}

            features = {}
            total_turns = len(timeline)
            # features['total_turns'] = total_turns

            # 初始化所有列表
            p1_hp_changes = []
            p2_hp_changes = []
            p1_moves = []
            p2_moves = []
            # 使用字典存储每个精灵的最后状态：{pokemon_name: status}
            p1_pokemon_status_dict = {}  # P1每个精灵的最后状态
            p2_pokemon_status_dict = {}  # P2每个精灵的最后状态
            p1_counter_invalid = 0  # Counter无效次数
            p2_counter_invalid = 0
            p1_move_powers = []
            p2_move_powers = []
            p1_move_accuracies = []
            p2_move_accuracies = []
            # 招式类别统计（已注释）
            # p1_physical_moves = 0
            # p2_physical_moves = 0
            # p1_special_moves = 0
            # p2_special_moves = 0
            # p1_status_moves = 0
            # p2_status_moves = 0
            p1_switch_count = 0
            p2_switch_count = 0
            # p1_move_null_count = 0  # P1 move_details为null的次数（保留用于兼容）
            # p2_move_null_count = 0  # P2 move_details为null的次数（保留用于兼容）
            p1_move_null_switch = 0  # P1因换人导致的move_details为null的次数
            p2_move_null_switch = 0  # P2因换人导致的move_details为null的次数
            p1_move_null_status = 0  # P1因状态异常导致的move_details为null的次数
            p2_move_null_status = 0  # P2因状态异常导致的move_details为null的次数
            
            # 初始化每个技能的使用次数计数器
            # p1_move_counts = {move: 0 for move in all_moves_list}
            # p2_move_counts = {move: 0 for move in all_moves_list}
            
            p1_boost_changes = []
            p2_boost_changes = []
            p1_final_boosts = {'atk': 0, 'def': 0, 'spa': 0, 'spd': 0, 'spe': 0}
            p2_final_boosts = {'atk': 0, 'def': 0, 'spa': 0, 'spd': 0, 'spe': 0}
            # effect_turns统计（已注释）
            # p1_effect_turns = 0
            # p2_effect_turns = 0
            # HP损失特征（已注释）
            p1_hp_losses = []
            p2_hp_losses = []

            # 分段统计
            early_end = total_turns // 3
            mid_end = 2 * total_turns // 3

            # 分段HP特征（已注释）
            # p1_early_hp = []
            # p2_early_hp = []
            # p1_mid_hp = []
            # p2_mid_hp = []
            # p1_late_hp = []
            # p2_late_hp = []

            # 连续攻击统计（已注释）
            # p1_consecutive_attacks = 0
            # p2_consecutive_attacks = 0
            # p1_max_consecutive = 0
            # p2_max_consecutive = 0
            # p1_last_was_attack = False
            # p2_last_was_attack = False

            # 换精灵时机
            p1_switch_turns = []
            p2_switch_turns = []
            
            # 前30回合出现的精灵统计
            p1_pokemon_appeared_30turns = set()  # P1在前30回合内出现的精灵集合
            p2_pokemon_appeared_30turns = set()  # P2在前30回合内出现的精灵集合
            
            # 存储每只精灵的hp_pct（前30回合）
            # 使用字典存储：{pokemon_name: [hp_pct_values]}
            p1_pokemon_hp_dict = {}  # P1每只精灵的hp_pct列表
            p2_pokemon_hp_dict = {}  # P2每只精灵的hp_pct列表

            # 用于跟踪前一轮的pokemon name，以判断换人
            p1_prev_pokemon_name = None
            p2_prev_pokemon_name = None
            # 用于HP损失特征：保存前一个回合的HP值
            p1_prev_hp = None
            p2_prev_hp = None

            for i, turn in enumerate(timeline):
                # HP数据（已注释 - 整体HP特征相关）
                # p1_hp = turn.get('p1_pokemon_state', {}).get('hp_pct', 0)
                # p2_hp = turn.get('p2_pokemon_state', {}).get('hp_pct', 0)
                # p1_hp_changes.append(p1_hp)
                # p2_hp_changes.append(p2_hp)
                
                # 为了HP损失特征，仍需要获取HP值
                p1_hp = turn.get('p1_pokemon_state', {}).get('hp_pct', 0)
                p2_hp = turn.get('p2_pokemon_state', {}).get('hp_pct', 0)
                
                # 统计前30回合出现的精灵和HP
                if i < 30:
                    p1_pokemon_name = turn.get('p1_pokemon_state', {}).get('name', '')
                    p2_pokemon_name = turn.get('p2_pokemon_state', {}).get('name', '')
                    
                    # 记录P1精灵出现和HP
                    if p1_pokemon_name:
                        p1_pokemon_appeared_30turns.add(p1_pokemon_name)
                        # 存储该精灵的hp_pct
                        if p1_pokemon_name not in p1_pokemon_hp_dict:
                            p1_pokemon_hp_dict[p1_pokemon_name] = []
                        p1_pokemon_hp_dict[p1_pokemon_name].append(p1_hp)
                    
                    # 记录P2精灵出现和HP
                    if p2_pokemon_name:
                        p2_pokemon_appeared_30turns.add(p2_pokemon_name)
                        # 存储该精灵的hp_pct
                        if p2_pokemon_name not in p2_pokemon_hp_dict:
                            p2_pokemon_hp_dict[p2_pokemon_name] = []
                        p2_pokemon_hp_dict[p2_pokemon_name].append(p2_hp)

                # 分段HP特征（已注释）
                # if i < early_end:
                #     p1_early_hp.append(p1_hp)
                #     p2_early_hp.append(p2_hp)
                # elif i < mid_end:
                #     p1_mid_hp.append(p1_hp)
                #     p2_mid_hp.append(p2_hp)
                # else:
                #     p1_late_hp.append(p1_hp)
                #     p2_late_hp.append(p2_hp)

                # HP损失
                if p1_prev_hp is not None:
                    p1_hp_loss = p1_prev_hp - p1_hp
                    p1_hp_losses.append(p1_hp_loss)
                if p2_prev_hp is not None:
                    p2_hp_loss = p2_prev_hp - p2_hp
                    p2_hp_losses.append(p2_hp_loss)
                
                # 更新前一个回合的HP值
                p1_prev_hp = p1_hp
                p2_prev_hp = p2_hp

                # 招式分析
                p1_move = turn.get('p1_move_details')
                p2_move = turn.get('p2_move_details')
                
                # 获取当前回合的pokemon name
                p1_current_pokemon_name = turn.get('p1_pokemon_state', {}).get('name', '')
                p2_current_pokemon_name = turn.get('p2_pokemon_state', {}).get('name', '')
                
                # 统计move_details为null的次数，并区分换人和状态异常
                # P1的null判断
                if p1_move is None:
                    # p1_move_null_count += 1  # 保留总数统计
                    # 判断是换人还是状态异常
                    if i > 0 and p1_prev_pokemon_name is not None and p1_current_pokemon_name != p1_prev_pokemon_name:
                        # 前一轮pokemon name和当前不一样，说明是换人
                        p1_move_null_switch += 1
                    else:
                        # pokemon name一样或第一轮，说明是状态异常（如睡眠、麻痹等）
                        p1_move_null_status += 1
                
                # P2的null判断
                if p2_move is None:
                    # p2_move_null_count += 1  # 保留总数统计
                    # 判断是换人还是状态异常
                    if i > 0 and p2_prev_pokemon_name is not None and p2_current_pokemon_name != p2_prev_pokemon_name:
                        # 前一轮pokemon name和当前不一样，说明是换人
                        p2_move_null_switch += 1
                    else:
                        # pokemon name一样或第一轮，说明是状态异常（如睡眠、麻痹等）
                        p2_move_null_status += 1
                
                # 更新前一轮的pokemon name（用于下一轮判断）
                p1_prev_pokemon_name = p1_current_pokemon_name
                p2_prev_pokemon_name = p2_current_pokemon_name
                
                # Counter技能分析 - 检测Counter是否无效
                # P1使用Counter时，检查P2的招式
                if p1_move and p1_move.get('name', '').lower() == 'counter':
                    # 如果P2没攻击（换人）或不是物理攻击，Counter无效
                    if not p2_move:  # P2换人
                        p1_counter_invalid += 1
                    elif p2_move.get('category', 'STATUS') != 'PHYSICAL':  # 不是物理攻击
                        p1_counter_invalid += 1
                
                # P2使用Counter时，检查P1的招式
                if p2_move and p2_move.get('name', '').lower() == 'counter':
                    # 如果P1没攻击（换人）或不是物理攻击，Counter无效
                    if not p1_move:  # P1换人
                        p2_counter_invalid += 1
                    elif p1_move.get('category', 'STATUS') != 'PHYSICAL':  # 不是物理攻击
                        p2_counter_invalid += 1

                # P1招式
                if p1_move:
                    move_name = p1_move.get('name', '')
                    p1_moves.append(move_name)
                    
                    # # 统计该技能的使用次数
                    # if move_name in p1_move_counts:
                    #     p1_move_counts[move_name] += 1
                    
                    power = p1_move.get('base_power', 0)
                    accuracy = p1_move.get('accuracy', 1.0)
                    category = p1_move.get('category', 'STATUS')

                    # 统计招式威力（连续攻击特征已注释）
                    # power > 0 表示这是一个攻击招式（物理或特殊），power = 0 表示状态招式
                    if power > 0:
                        # 当前是攻击招式
                        p1_move_powers.append(power)  # 记录招式威力，用于后续计算平均/最大/最小威力
                        # 连续攻击统计（已注释）
                        # p1_consecutive_attacks += 1   # 连续攻击计数+1
                        # p1_last_was_attack = True      # 标记当前回合是攻击
                    # else:
                        # 当前是状态招式（power = 0）
                        # 连续攻击统计（已注释）
                        # if p1_last_was_attack:
                        #     # 如果上一回合是攻击，说明连续攻击刚刚结束
                        #     # 更新最大连续攻击次数（记录到目前为止最长的连续攻击序列）
                        #     p1_max_consecutive = max(p1_max_consecutive, p1_consecutive_attacks)
                        #     p1_consecutive_attacks = 0  # 重置连续攻击计数器
                        # p1_last_was_attack = False     # 标记当前回合不是攻击

                    p1_move_accuracies.append(accuracy)

                    # 招式类别统计（已注释）
                    # if category == 'PHYSICAL':
                    #     p1_physical_moves += 1
                    # elif category == 'SPECIAL':
                    #     p1_special_moves += 1
                    # elif category == 'STATUS':
                    #     p1_status_moves += 1
                else:
                    p1_switch_count += 1
                    p1_switch_turns.append(i)
                    # 连续攻击统计（已注释）
                    # if p1_last_was_attack:
                    #     p1_max_consecutive = max(p1_max_consecutive, p1_consecutive_attacks)
                    #     p1_consecutive_attacks = 0
                    # p1_last_was_attack = False

                # P2招式
                if p2_move:
                    move_name = p2_move.get('name', '')
                    p2_moves.append(move_name)
                    
                    # # 统计该技能的使用次数
                    # if move_name in p2_move_counts:
                    #     p2_move_counts[move_name] += 1
                    
                    power = p2_move.get('base_power', 0)
                    accuracy = p2_move.get('accuracy', 1.0)
                    category = p2_move.get('category', 'STATUS')

                    if power > 0:
                        p2_move_powers.append(power)
                        # 连续攻击统计（已注释）
                        # p2_consecutive_attacks += 1
                        # p2_last_was_attack = True
                    # else:
                        # 连续攻击统计（已注释）
                        # if p2_last_was_attack:
                        #     p2_max_consecutive = max(p2_max_consecutive, p2_consecutive_attacks)
                        #     p2_consecutive_attacks = 0
                        # p2_last_was_attack = False

                    p2_move_accuracies.append(accuracy)

                    # 招式类别统计（已注释）
                    # if category == 'PHYSICAL':
                    #     p2_physical_moves += 1
                    # elif category == 'SPECIAL':
                    #     p2_special_moves += 1
                    # elif category == 'STATUS':
                    #     p2_status_moves += 1
                else:
                    p2_switch_count += 1
                    p2_switch_turns.append(i)
                    # 连续攻击统计（已注释）
                    # if p2_last_was_attack:
                    #     p2_max_consecutive = max(p2_max_consecutive, p2_consecutive_attacks)
                    #     p2_consecutive_attacks = 0
                    # p2_last_was_attack = False

                # 状态变化 - 存储每个精灵的最后状态
                p1_pokemon_name = turn.get('p1_pokemon_state', {}).get('name', '')
                p1_status = turn.get('p1_pokemon_state', {}).get('status', 'nostatus')
                if p1_pokemon_name:
                    # 只保存某个精灵的最后一次出现的状态
                    p1_pokemon_status_dict[p1_pokemon_name] = p1_status

                p2_pokemon_name = turn.get('p2_pokemon_state', {}).get('name', '')
                p2_status = turn.get('p2_pokemon_state', {}).get('status', 'nostatus')
                if p2_pokemon_name:
                    # 只保存某个精灵的最后一次出现的状态
                    p2_pokemon_status_dict[p2_pokemon_name] = p2_status

                # 属性提升
                p1_boosts = turn.get('p1_pokemon_state', {}).get('boosts', {})
                p2_boosts = turn.get('p2_pokemon_state', {}).get('boosts', {})

                if p1_boosts:
                    p1_final_boosts = p1_boosts
                    boost_sum = sum(p1_boosts.values())
                    p1_boost_changes.append(boost_sum)

                if p2_boosts:
                    p2_final_boosts = p2_boosts
                    boost_sum = sum(p2_boosts.values())
                    p2_boost_changes.append(boost_sum)


            # HP损失特征 好像还确实有用
            if p1_hp_losses:
                features['p1_avg_hp_loss'] = np.mean(p1_hp_losses)
                features['p1_max_hp_loss'] = max(p1_hp_losses)
            else:
                features['p1_avg_hp_loss'] = 0
                features['p1_max_hp_loss'] = 0

            if p2_hp_losses:
                features['p2_avg_hp_loss'] = np.mean(p2_hp_losses)
                features['p2_max_hp_loss'] = max(p2_hp_losses)
            else:
                features['p2_avg_hp_loss'] = 0
                features['p2_max_hp_loss'] = 0

            # 统计前30回合出现的精灵中有异常状态（非nostatus）的精灵数量
            # 获取前30回合出现的所有精灵（最多6只，也可能不足6只）
            p1_abnormal_status_count = 0
            for pokemon_name in p1_pokemon_appeared_30turns:
                # 获取该精灵的最后状态
                status = p1_pokemon_status_dict.get(pokemon_name, 'nostatus')
                if status != 'nostatus' and status != 'fnt':
                    p1_abnormal_status_count += 1
            
            p2_abnormal_status_count = 0
            for pokemon_name in p2_pokemon_appeared_30turns:
                # 获取该精灵的最后状态
                status = p2_pokemon_status_dict.get(pokemon_name, 'nostatus')
                if status != 'nostatus' and status != 'fnt':
                    p2_abnormal_status_count += 1
            
            # 计算异常状态数量比值（P1 / P2，使用平滑处理避免除以0）
            features['p1_abnormal_status_count'] = p1_abnormal_status_count
            features['p2_abnormal_status_count'] = p2_abnormal_status_count
            features['abnormal_status_count_ratio'] = p1_abnormal_status_count / (p2_abnormal_status_count + 1.0)
            
            # Counter技能无效次数（第一世代机制）
            features['p1_counter_invalid'] = p1_counter_invalid  # P1使用Counter但无效的次数
            features['p2_counter_invalid'] = p2_counter_invalid  # P2使用Counter但无效的次数

            # 命中率
            if p1_move_accuracies:
                features['p1_avg_accuracy'] = np.mean(p1_move_accuracies)
            else:
                features['p1_avg_accuracy'] = 1.0

            if p2_move_accuracies:
                features['p2_avg_accuracy'] = np.mean(p2_move_accuracies)
            else:
                features['p2_avg_accuracy'] = 1.0


            # 换精灵特征
            features['p1_switch_count'] = p1_switch_count
            features['p2_switch_count'] = p2_switch_count
            
            # move_details为null的次数（换人或未使用招式）
            # features['p1_move_null_count'] = p1_move_null_count  # P1 move_details为null的总次数（保留用于兼容）
            # features['p2_move_null_count'] = p2_move_null_count  # P2 move_details为null的总次数（保留用于兼容）
            # 区分换人和状态异常导致的null
            features['p1_move_null_switch'] = p1_move_null_switch  # P1因换人导致的move_details为null的次数
            features['p2_move_null_switch'] = p2_move_null_switch  # P2因换人导致的move_details为null的次数
            features['p1_move_null_status'] = p1_move_null_status  # P1因状态异常导致的move_details为null的次数
            features['p2_move_null_status'] = p2_move_null_status  # P2因状态异常导致的move_details为null的次数
            
            # 每个技能的使用次数统计（P1和P2各40个技能）
            # for move_name in all_moves_list:
            #     feature_name_p1 = f'p1_move_{move_to_feature[move_name]}_count'
            #     feature_name_p2 = f'p2_move_{move_to_feature[move_name]}_count'
            #     features[feature_name_p1] = p1_move_counts.get(move_name, 0)
            #     features[feature_name_p2] = p2_move_counts.get(move_name, 0)

            # 换精灵时机(早中晚期占比)
            if p1_switch_turns:
                early_switches = sum(1 for t in p1_switch_turns if t < early_end)
                features['p1_early_switch_ratio'] = early_switches / max(p1_switch_count, 1)
            else:
                features['p1_early_switch_ratio'] = 0

            if p2_switch_turns:
                early_switches = sum(1 for t in p2_switch_turns if t < early_end)
                features['p2_early_switch_ratio'] = early_switches / max(p2_switch_count, 1)
            else:
                features['p2_early_switch_ratio'] = 0


            
            
            # 前30回合出现的精灵个数统计
            features['p1_unique_pokemon_count_30turns'] = len(p1_pokemon_appeared_30turns)
            features['p2_unique_pokemon_count_30turns'] = len(p2_pokemon_appeared_30turns)
            
            # 计算P1和P2前30回合加权强度值（hp_pct * 强度值）
            # 对于已出现的精灵：使用实际的hp_pct和strength
            # 对于未出现的精灵（不足6只时）：hp_pct按1.0计算，strength按已出现精灵的均值计算
            # 统一从pokemon_db中查找所有精灵的属性数据
            
            # P1加权强度值计算
            p1_weighted_strength_sum = 0.0
            p1_strength_list = []  # 收集已出现精灵的强度值，用于计算均值
            for pokemon_name, hp_list in p1_pokemon_hp_dict.items():
                if hp_list:  # 如果该精灵有hp记录
                    # 取最后一次出现的hp_pct
                    last_hp = hp_list[-1]
                    # 只计算hp_pct > 0的精灵
                    if last_hp > 0:
                        # 从pokemon_db中查找该精灵的属性数据
                        pokemon_name_lower = pokemon_name.lower()
                        if pokemon_name_lower in pokemon_db:
                            pokemon_data = pokemon_db[pokemon_name_lower]
                            # 计算强度值
                            strength = calculate_pokemon_strength(pokemon_data)
                            # 加权强度值 = hp_pct * 强度值
                            p1_weighted_strength_sum += last_hp * strength
                            # 收集强度值用于计算均值
                            p1_strength_list.append(strength)
                        # 如果找不到属性数据，跳过该精灵（无法计算强度值）
            
            # 如果不足6只精灵，剩余精灵按hp_pct=1.0，strength=已出现精灵的均值计算
            p1_appeared_count = len(p1_pokemon_hp_dict)
            p1_missing_count = 6 - p1_appeared_count
            if p1_missing_count > 0 and len(p1_strength_list) > 0:
                # 计算已出现精灵的强度值均值
                p1_avg_strength = np.mean(p1_strength_list)
                # 未出现精灵的贡献：hp_pct=1.0 * 平均强度值
                p1_weighted_strength_sum += p1_missing_count * 1.0 * p1_avg_strength
            
            # P2加权强度值计算
            p2_weighted_strength_sum = 0.0
            p2_strength_list = []  # 收集已出现精灵的强度值，用于计算均值
            for pokemon_name, hp_list in p2_pokemon_hp_dict.items():
                if hp_list:  # 如果该精灵有hp记录
                    # 取最后一次出现的hp_pct
                    last_hp = hp_list[-1]
                    # 只计算hp_pct > 0的精灵
                    if last_hp > 0:
                        # 从pokemon_db中查找该精灵的属性数据
                        pokemon_name_lower = pokemon_name.lower()
                        if pokemon_name_lower in pokemon_db:
                            pokemon_data = pokemon_db[pokemon_name_lower]
                            # 计算强度值
                            strength = calculate_pokemon_strength(pokemon_data)
                            # 加权强度值 = hp_pct * 强度值
                            p2_weighted_strength_sum += last_hp * strength
                            # 收集强度值用于计算均值
                            p2_strength_list.append(strength)
                        # 如果找不到属性数据，跳过该精灵（无法计算强度值）
            
            # 如果不足6只精灵，剩余精灵按hp_pct=1.0，strength=已出现精灵的均值计算
            p2_appeared_count = len(p2_pokemon_hp_dict)
            p2_missing_count = 6 - p2_appeared_count
            if p2_missing_count > 0 and len(p2_strength_list) > 0:
                # 计算已出现精灵的强度值均值
                p2_avg_strength = np.mean(p2_strength_list)
                # 未出现精灵的贡献：hp_pct=1.0 * 平均强度值
                p2_weighted_strength_sum += p2_missing_count * 1.0 * p2_avg_strength
            
            features['p1_weighted_strength_sum'] = p1_weighted_strength_sum
            features['p2_weighted_strength_sum'] = p2_weighted_strength_sum
            # 计算P1和P2加权强度值的比值
            if p2_weighted_strength_sum > 0:
                features['weighted_strength_ratio_30turns'] = p1_weighted_strength_sum / p2_weighted_strength_sum
            else:
                features['weighted_strength_ratio_30turns'] = 0.0  # 避免除以0
            
            # 统计P1和P2的fnt（死亡）数量
            # 统计所有在时间线中出现过的精灵中，状态为'fnt'的数量
            p1_fnt_count = 0
            for pokemon_name, status in p1_pokemon_status_dict.items():
                if status == 'fnt':
                    p1_fnt_count += 1
            
            p2_fnt_count = 0
            for pokemon_name, status in p2_pokemon_status_dict.items():
                if status == 'fnt':
                    p2_fnt_count += 1
            
            # 计算fnt数量比值（p1_fnt_count / p2_fnt_count）
            # 使用平滑处理避免除以0：p1_fnt_count / (p2_fnt_count + 1)
            # 这样当p2_fnt_count为0时，比值 = p1_fnt_count，仍能反映P1的损失情况
            features['fnt_count_ratio'] = p1_fnt_count / (p2_fnt_count + 1.0)
            
            # 添加差值特征作为补充（反映绝对优势）
            features['fnt_count_diff'] = p1_fnt_count - p2_fnt_count
            
            # 第30回合组合胜率特征 ⭐新增
            # 提取第30回合时双方存活的宝可梦组合，并匹配历史胜率
            # turn30_combination_winrate = 1.0  # 默认值（中性，表示无历史数据）
            # 
            # if False:#total_turns >= 30 and turn30_winrates:
            #     # 截取前30回合数据，获取双方在第30回合时的存活精灵集合
            #     first_30_turns = timeline[:30]
            #     (
            #         p1_turn30_alive_raw,
            #         p1_turn30_fnt_raw,
            #         p2_turn30_alive_raw,
            #     ) = extract_final_pokemon_status(first_30_turns)
            #     
            #     # 从p1_team中提取未死亡的精灵名称，然后与p1_alive_pokemon_names合并
            #     # 这样既包含未出场的存活精灵，也包含时间线中出现的存活精灵
            #     p1_team = row.get('p1_team_details', [])
            #     p1_team_names = {pokemon.get('name', '') for pokemon in p1_team 
            #                 if pokemon and pokemon.get('name', '') not in p1_turn30_fnt_raw}
            #     p1_turn30_alive_raw = p1_team_names | p1_turn30_alive_raw
            # 
            #     p1_turn30_alive = {name.lower() for name in p1_turn30_alive_raw}
            #     p2_turn30_alive = {name.lower() for name in p2_turn30_alive_raw}
            # 
            #     # 前边已经提取过了，不要重复
            #     # 如果双方都有存活宝可梦，尝试匹配历史胜率
            #     if p1_turn30_alive and p2_turn30_alive:
            #         # 创建组合键：将宝可梦集合排序后连接
            #         # 格式：p1_pokemon1,p1_pokemon2|p2_pokemon1,p2_pokemon2
            #         p1_sorted = ','.join(sorted(p1_turn30_alive))
            #         p2_sorted = ','.join(sorted(p2_turn30_alive))
            #         combination_key = f"{p1_sorted}|{p2_sorted}"
            #         
            #         # 在胜率数据库中查找匹配的组合
            #         if combination_key in turn30_winrates:
            #             # 找到完全匹配，使用历史胜率比值
            #             turn30_combination_winrate = turn30_winrates[combination_key].get('ratio', 1.0)
            #         # 如果没有完全匹配，保持默认值1.0（中性）
            # 
            # features['turn30_combination_winrate'] = turn30_combination_winrate

            # 特征：类型克制效果倍数（p1_avg, p2_avg）⭐新增
            # 提取战斗过程中每个玩家使用的技能集合
            battle_data = {'battle_timeline': timeline}
            p1_moves, p2_moves = extract_moves(battle_data)
            
            # 计算每个队伍对对方队伍的平均类型效果倍数
            p1_team_avg, p2_team_avg = type_multiplier(p1_moves, p2_moves)
            features['p1_avg'] = p1_team_avg  # 玩家1对玩家2的类型效果倍数
            features['p2_avg'] = p2_team_avg  # 玩家2对玩家1的类型效果倍数
            
            # 特征：先制技能优先级总和（p1_num_priority_moves, p2_num_priority_moves）⭐新增
            # 统计所有先制技能的优先级值总和
            features['p1_num_priority_moves'] = count_priority_moves(p1_moves)  # 玩家1先制技能优先级总和
            features['p2_num_priority_moves'] = count_priority_moves(p2_moves)  # 玩家2先制技能优先级总和

            return features

        # 为训练和测试数据提取特征
        train_battle_features = []
        for idx, row in self.train_data.iterrows(): # idx == 238
            timeline = row.get('battle_timeline', [])
            battle_features = analyze_battle_timeline(row, timeline, pokemon_db, None)  # self.turn30_winrates
            train_battle_features.append(battle_features)

        test_battle_features = []
        for idx, row in self.test_data.iterrows():
            timeline = row.get('battle_timeline', [])
            battle_features = analyze_battle_timeline(row, timeline, pokemon_db, None)  # self.turn30_winrates
            test_battle_features.append(battle_features)

        self.train_dynamic_features = pd.DataFrame(train_battle_features)
        self.test_dynamic_features = pd.DataFrame(test_battle_features)

        print(f"动态特征提取完成: {self.train_dynamic_features.shape[1]} 个特征")
        return self.train_dynamic_features, self.test_dynamic_features

    def create_interaction_features(self, df):
        """
        创建特征交互项
        
        共4个交互特征：
        - hp_boost_interaction: HP优势 × Boost优势 (血量和属性提升协同)
        - p1_effective_power: P1威力 × P1命中率 (有效攻击力)
        - p2_effective_power: P2威力 × P2命中率 (有效攻击力)
        - type_stats_interaction: 类型优势 × 属性优势 (克制和实力的组合)
        """
        df_copy = df.copy()

        # 关键特征组合
        # HP优势交互特征（已注释 - 依赖整体HP特征）
        # if 'hp_advantage_end' in df_copy.columns and 'boost_advantage' in df_copy.columns:
        #     df_copy['hp_boost_interaction'] = df_copy['hp_advantage_end'] * df_copy['boost_advantage']

        if 'p1_avg_move_power' in df_copy.columns and 'p1_avg_accuracy' in df_copy.columns:
            df_copy['p1_effective_power'] = df_copy['p1_avg_move_power'] * df_copy['p1_avg_accuracy']

        if 'p2_avg_move_power' in df_copy.columns and 'p2_avg_accuracy' in df_copy.columns:
            df_copy['p2_effective_power'] = df_copy['p2_avg_move_power'] * df_copy['p2_avg_accuracy']

        if 'type_adv_mean' in df_copy.columns and 'total_stats_advantage' in df_copy.columns:
            df_copy['type_stats_interaction'] = df_copy['type_adv_mean'] * df_copy['total_stats_advantage']

        return df_copy

    def combine_features(self):
        """合并所有特征并创建交互项"""
        print("\n=== 合并特征并创建交互项 ===")

        # 合并基础特征
        self.train_combined = pd.concat([
            self.train_static_features.reset_index(drop=True),
            self.train_dynamic_features.reset_index(drop=True)
        ], axis=1)

        self.test_combined = pd.concat([
            self.test_static_features.reset_index(drop=True),
            self.test_dynamic_features.reset_index(drop=True)
        ], axis=1)

        # 创建交互特征
        self.train_combined = self.create_interaction_features(self.train_combined)
        self.test_combined = self.create_interaction_features(self.test_combined)

        # 根据配置列表移除特征
        if self.features_to_remove:
            print(f"\n=== 移除配置的特征 ({len(self.features_to_remove)}个) ===")
            features_to_remove_actual = []
            for feature in self.features_to_remove:
                if feature in self.train_combined.columns:
                    features_to_remove_actual.append(feature)
                else:
                    print(f"  警告: 特征 '{feature}' 不存在，跳过移除")
            
            if features_to_remove_actual:
                self.train_combined = self.train_combined.drop(columns=features_to_remove_actual)
                self.test_combined = self.test_combined.drop(columns=features_to_remove_actual)
                print(f"  ✓ 已移除 {len(features_to_remove_actual)} 个特征: {features_to_remove_actual}")
            else:
                print("  ⚠️ 没有找到要移除的特征")

        # 处理缺失值
        self.train_combined = self.train_combined.fillna(0)
        self.test_combined = self.test_combined.fillna(0)

        # 确保两个数据集有相同的列
        train_cols = set(self.train_combined.columns)
        test_cols = set(self.test_combined.columns)

        missing_in_test = train_cols - test_cols
        missing_in_train = test_cols - train_cols

        for col in missing_in_test:
            self.test_combined[col] = 0
        for col in missing_in_train:
            self.train_combined[col] = 0

        # 对齐列顺序
        self.test_combined = self.test_combined[self.train_combined.columns]

        print(f"特征合并完成:")
        print(f"训练特征形状: {self.train_combined.shape}")
        print(f"测试特征形状: {self.test_combined.shape}")

        return self.train_combined, self.test_combined

    def select_features_rfecv(self, X_train, y_train, estimator=None, cv=5, scoring='accuracy', min_features_to_select=10, n_jobs=-1):
        """
        使用RFECV（递归特征消除与交叉验证）进行特征选择
        
        参数:
        - X_train: 训练特征
        - y_train: 训练标签
        - estimator: 用于特征选择的估计器（默认使用LogisticRegression）
        - cv: 交叉验证折数（默认5折）
        - scoring: 评分标准（默认'accuracy'）
        - min_features_to_select: 最少保留的特征数（默认10）
        - n_jobs: 并行任务数（默认-1，使用所有CPU）
        
        返回:
        - X_train_selected: 选择后的训练特征
        - selected_features: 选中的特征名称列表
        - optimal_n_features: 最优特征数量
        """
        print(f"\n=== RFECV特征选择 ===")
        print(f"初始特征数量: {X_train.shape[1]}")
        print(f"交叉验证折数: {cv}")
        print(f"最少保留特征数: {min_features_to_select}")
        print("正在计算最优特征数量（这可能需要几分钟）...")
        
        # 如果没有提供估计器，使用XGBoost作为默认估计器
        # 注意：RFECV会多次训练模型（每个特征数量都要训练cv次），所以使用轻量级参数以加快速度
        # 特征选择的目标是找到重要特征，不需要模型达到最佳性能
        if estimator is None:
            estimator = xgb.XGBClassifier(
                n_estimators=100,  # 减少树数量，加快特征选择速度（从800降到100）
                max_depth=6,  # 稍微降低深度，加快训练（从8降到6）
                learning_rate=0.1,  # 提高学习率，补偿树数量减少（从0.03提高到0.1）
                subsample=0.8,  # 稍微降低采样率，加快训练（从0.85降到0.8）
                colsample_bytree=0.8,  # 稍微降低列采样率（从0.85降到0.8）
                min_child_weight=1,  # 简化参数（从2降到1）
                gamma=0,  # 简化参数，加快训练（从0.05降到0）
                reg_alpha=0.01,  # 降低正则化，加快训练（从0.1降到0.01）
                reg_lambda=0.01,  # 降低正则化，加快训练（从0.1降到0.01）
                random_state=42,
                eval_metric='logloss',
                n_jobs=n_jobs
            )
        
        # 创建RFECV对象
        rfecv = RFECV(
            estimator=estimator,
            step=1,  # 每次移除1个特征
            cv=cv,
            scoring=scoring,
            min_features_to_select=min_features_to_select,
            n_jobs=n_jobs
        )
        
        # 执行特征选择
        rfecv.fit(X_train, y_train)
        
        # 保存特征选择器供后续使用
        self.feature_selector = rfecv
        
        # 获取选中的特征
        selected_mask = rfecv.support_
        selected_features = X_train.columns[selected_mask].tolist()
        optimal_n_features = rfecv.n_features_
        
        # 转换数据
        X_train_selected = rfecv.transform(X_train)
        
        # 如果输入是DataFrame，将结果也转换为DataFrame以保持一致性
        if isinstance(X_train, pd.DataFrame):
            X_train_selected = pd.DataFrame(
                X_train_selected,
                columns=selected_features,
                index=X_train.index
            )
        
        # 获取被移除的特征
        all_features = X_train.columns.tolist()
        removed_features = [feat for feat in all_features if feat not in selected_features]
        
        print(f"✓ RFECV特征选择完成")
        print(f"  最优特征数量: {optimal_n_features}")
        print(f"  选中的特征数量: {len(selected_features)}")
        print(f"  特征减少: {X_train.shape[1] - optimal_n_features} 个 ({(1 - optimal_n_features/X_train.shape[1])*100:.1f}%)")
        
        # 打印被移除的特征
        if removed_features:
            print(f"\n被移除的特征 ({len(removed_features)} 个):")
            # 按特征名称排序以便查看
            removed_features_sorted = sorted(removed_features)
            # 每行显示5个特征，便于阅读
            for i in range(0, len(removed_features_sorted), 5):
                features_line = removed_features_sorted[i:i+5]
                print(f"  {', '.join(features_line)}")
        else:
            print(f"\n没有特征被移除")
        
        # 显示交叉验证得分随特征数量的变化
        print(f"\n交叉验证得分随特征数量的变化:")
        print(f"  最高得分: {rfecv.cv_results_['mean_test_score'].max():.4f}")
        print(f"  最优特征数时的得分: {rfecv.cv_results_['mean_test_score'][rfecv.n_features_ - min_features_to_select]:.4f}")
        
        return X_train_selected, selected_features, optimal_n_features

    def save_misclassified_samples(self, y_true_indices, y_pred, y_true, model_name='best_model'):
        """
        保存预测错误的样本到JSON文件
        
        功能：
        - 识别验证集中预测错误的样本
        - 保存完整的战斗记录（包含时间线、队伍信息等）
        - 添加预测信息标签（predicted, actual, model, index）
        - 生成统计信息（准确率、假阳性、假阴性等）
        
        输出文件：
        - misclassified_samples_{model_name}.json: 错误样本详情
        - misclassified_stats_{model_name}.json: 统计信息
        """
        print(f"\n=== 保存{model_name}的预测错误样本 ===")
        
        # 找出预测错误的索引位置
        misclassified_mask = y_pred != y_true.values
        misclassified_indices = y_true_indices[misclassified_mask]
        
        if len(misclassified_indices) == 0:
            print("没有预测错误的样本！")
            return None, None
        
        print(f"发现 {len(misclassified_indices)} 个预测错误的样本")
        
        # 获取错误预测的值
        y_pred_wrong = y_pred[misclassified_mask]
        y_true_wrong = y_true.values[misclassified_mask]
        
        # 从原始训练数据中提取错误样本
        misclassified_samples = []
        for i, idx in enumerate(misclassified_indices):
            sample = self.train_data.iloc[idx].to_dict()
            
            # 添加预测信息
            sample['prediction_info'] = {
                'predicted': int(y_pred_wrong[i]),
                'actual': int(y_true_wrong[i]),
                'model': model_name,
                'original_index': int(idx)
            }
            
            misclassified_samples.append(sample)
        
        # 生成带时间戳的文件名
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        output_file = f'misclassified/misclassified_samples_{model_name}_{timestamp}.json'
        
        # 保存到JSON文件
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(misclassified_samples, f, indent=2, ensure_ascii=False)
        
        print(f"预测错误样本已保存到: {output_file}")
        
        # 保存简单统计信息
        stats = {
            'total_samples': len(y_true),
            'misclassified_count': len(misclassified_indices),
            'accuracy': 1 - (len(misclassified_indices) / len(y_true)),
            'error_rate': len(misclassified_indices) / len(y_true),
            'false_positive': int(((y_pred == 1) & (y_true == 0)).sum()),
            'false_negative': int(((y_pred == 0) & (y_true == 1)).sum()),
            'timestamp': timestamp,
            'model_name': model_name
        }
        
        stats_file = f'misclassified/misclassified_stats_{model_name}_{timestamp}.json'
        with open(stats_file, 'w', encoding='utf-8') as f:
            json.dump(stats, f, indent=2)
        
        print(f"统计信息已保存到: {stats_file}")
        print(f"准确率: {stats['accuracy']:.4f}")
        print(f"错误率: {stats['error_rate']:.4f}")
        print(f"假阳性(预测赢但实际输): {stats['false_positive']}")
        print(f"假阴性(预测输但实际赢): {stats['false_negative']}")
        
        return misclassified_samples, stats

    # ========================================================================
    # Optuna超参数优化相关方法
    # ========================================================================

    def optimize_xgboost_hyperparams(self, X_train, y_train):
        """
        使用Optuna + MedianPruner优化XGBoost超参数

        参数:
        - X_train: 训练特征
        - y_train: 训练标签

        返回:
        - best_params: 最优超参数字典
        """
        print(f"\n=== XGBoost超参数优化 ===")
        print(f"试验次数: {self.optuna_n_trials}")
        print(f"交叉验证折数: {self.optuna_cv_folds}")
        print(f"超时时间: {self.optuna_timeout}秒")

        # 定义目标函数：Optuna会多次调用此函数，每次传入不同的trial对象
        def objective(trial):
            """
            Optuna目标函数：评估一组超参数的性能
            
            参数:
            - trial: Optuna的Trial对象，用于建议超参数值和报告中间结果
            
            返回:
            - 交叉验证的平均准确率（Optuna会最大化此值）
            """
            # ========================================================================
            # XGBoost超参数搜索空间定义
            # ========================================================================
            # 使用trial.suggest_*方法定义每个超参数的搜索范围
            # Optuna会根据历史试验结果智能选择下一个要尝试的参数组合
            params = {
                # 树的数量：200-1000，步长100（即200, 300, 400, ..., 1000）
                'n_estimators': trial.suggest_int('n_estimators', 200, 1000, step=100),
                
                # 树的最大深度：4-12（控制模型复杂度，防止过拟合）
                'max_depth': trial.suggest_int('max_depth', 4, 12),
                
                # 学习率：0.01-0.3，对数尺度（log=True表示在对数空间均匀采样）
                # 较小的学习率通常需要更多树，但可能获得更好的泛化性能
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                
                # 行采样比例：0.6-1.0（每棵树使用的训练样本比例，防止过拟合）
                'subsample': trial.suggest_float('subsample', 0.6, 1.0),
                
                # 列采样比例：0.6-1.0（每棵树使用的特征比例，增加随机性）
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
                
                # 叶子节点最小样本权重和：1-10（控制叶子节点分裂，值越大越保守）
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
                
                # 最小损失减少量：0-0.5（分裂所需的最小增益，值越大越保守）
                'gamma': trial.suggest_float('gamma', 0, 0.5),
                
                # L1正则化系数：0.001-1.0，对数尺度（控制特征选择，防止过拟合）
                'reg_alpha': trial.suggest_float('reg_alpha', 0.001, 1.0, log=True),
                
                # L2正则化系数：0.001-1.0，对数尺度（控制权重大小，防止过拟合）
                'reg_lambda': trial.suggest_float('reg_lambda', 0.001, 1.0, log=True),
                
                # 固定参数（不参与优化）
                'random_state': 42,  # 随机种子，保证可复现性
                'eval_metric': 'logloss',  # 评估指标
                'n_jobs': -1  # 使用所有CPU核心
            }

            # 使用建议的超参数创建XGBoost模型
            model = xgb.XGBClassifier(**params)

            # ========================================================================
            # 交叉验证评估超参数性能
            # ========================================================================
            # 使用分层K折交叉验证，保证每折中各类别比例一致
            cv = StratifiedKFold(n_splits=self.optuna_cv_folds, shuffle=True, random_state=42)
            scores = []  # 存储每折的准确率

            # 遍历每个交叉验证折
            for fold_idx, (train_idx, val_idx) in enumerate(cv.split(X_train, y_train)):
                # 处理DataFrame和numpy数组两种数据格式
                X_fold_train = X_train.iloc[train_idx] if hasattr(X_train, 'iloc') else X_train[train_idx]
                y_fold_train = y_train.iloc[train_idx] if hasattr(y_train, 'iloc') else y_train[train_idx]
                X_fold_val = X_train.iloc[val_idx] if hasattr(X_train, 'iloc') else X_train[val_idx]
                y_fold_val = y_train.iloc[val_idx] if hasattr(y_train, 'iloc') else y_train[val_idx]

                # 在当前折的训练集上训练模型
                model.fit(X_fold_train, y_fold_train)

                # 在验证集上预测并计算准确率
                y_pred = model.predict(X_fold_val)
                fold_accuracy = accuracy_score(y_fold_val, y_pred)
                scores.append(fold_accuracy)

                # ========================================================================
                # Pruning（剪枝）机制：提前终止表现差的试验
                # ========================================================================
                # 报告当前fold的中间结果（准确率）和步数（fold索引）
                # MedianPruner会根据已完成的试验的中位数性能决定是否剪枝
                trial.report(fold_accuracy, fold_idx)

                # 检查是否应该剪枝：如果当前试验表现明显低于中位数，提前终止
                # 这样可以节省计算资源，将时间投入到更有希望的参数组合上
                if trial.should_prune():
                    raise optuna.TrialPruned()  # 抛出异常，终止当前试验

            # 返回所有折的平均准确率（Optuna会最大化此值）
            return np.mean(scores)

        # ========================================================================
        # 创建MedianPruner（中位数剪枝器）
        # ========================================================================
        # MedianPruner会提前终止表现低于中位数的试验，节省计算资源
        # n_startup_trials: 前N个试验不进行剪枝（预热期），让Pruner有足够数据建立基准
        # n_warmup_steps: 每个试验的前N步不剪枝，给模型足够的训练时间
        pruner = MedianPruner(
            n_startup_trials=self.optuna_pruner_warmup,  # 前5个试验不剪枝
            n_warmup_steps=self.optuna_pruner_interval  # 每个试验前1步不剪枝
        )

        # ========================================================================
        # 创建Optuna Study并开始优化
        # ========================================================================
        # 设置日志级别为WARNING，减少输出信息（只显示警告和错误）
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        # 创建Study对象：管理整个超参数优化过程
        study = optuna.create_study(
            direction='maximize',  # 最大化目标函数（准确率）
            pruner=pruner,  # 使用MedianPruner进行早期剪枝
            study_name='xgboost_optimization'  # Study名称，用于标识和保存
        )

        # 开始优化：Optuna会智能地选择超参数组合进行试验
        # 使用TPE（Tree-structured Parzen Estimator）算法进行贝叶斯优化
        study.optimize(
            objective,  # 目标函数
            n_trials=self.optuna_n_trials,  # 最大试验次数（50次）
            timeout=self.optuna_timeout,  # 超时时间（3600秒=1小时）
            show_progress_bar=True  # 显示进度条
        )

        # ========================================================================
        # 保存优化结果
        # ========================================================================
        # 保存Study对象和最优参数，供后续使用和保存到文件
        self.optuna_studies['XGBoost'] = study
        self.optuna_best_params['XGBoost'] = study.best_params

        # ========================================================================
        # 打印优化结果摘要
        # ========================================================================
        print(f"\n✓ XGBoost优化完成")
        print(f"  最优准确率: {study.best_value:.4f}")  # 所有试验中的最高准确率
        print(f"  最优参数:")  # 达到最优准确率的超参数组合
        for key, value in study.best_params.items():
            print(f"    {key}: {value}")
        print(f"  完成试验: {len(study.trials)}")  # 成功完成的试验数
        # 统计被剪枝的试验数（提前终止的试验）
        print(f"  剪枝试验: {len([t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED])}")

        return study.best_params

    def optimize_lightgbm_hyperparams(self, X_train, y_train):
        """
        使用Optuna + MedianPruner优化LightGBM超参数

        参数:
        - X_train: 训练特征
        - y_train: 训练标签

        返回:
        - best_params: 最优超参数字典
        """
        print(f"\n=== LightGBM超参数优化 ===")
        print(f"试验次数: {self.optuna_n_trials}")
        print(f"交叉验证折数: {self.optuna_cv_folds}")
        print(f"超时时间: {self.optuna_timeout}秒")

        # 定义目标函数：Optuna会多次调用此函数，每次传入不同的trial对象
        def objective(trial):
            """
            Optuna目标函数：评估一组超参数的性能
            
            参数:
            - trial: Optuna的Trial对象，用于建议超参数值和报告中间结果
            
            返回:
            - 交叉验证的平均准确率（Optuna会最大化此值）
            """
            # ========================================================================
            # LightGBM超参数搜索空间定义
            # ========================================================================
            # LightGBM特有的参数：num_leaves（叶子节点数）和min_child_samples（叶子节点最小样本数）
            params = {
                # 树的数量：200-1000，步长100
                'n_estimators': trial.suggest_int('n_estimators', 200, 1000, step=100),
                
                # 树的最大深度：4-12
                'max_depth': trial.suggest_int('max_depth', 4, 12),
                
                # 学习率：0.01-0.3，对数尺度
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                
                # 叶子节点数：20-100（LightGBM特有，控制模型复杂度）
                # 注意：num_leaves应小于2^max_depth，否则可能过拟合
                'num_leaves': trial.suggest_int('num_leaves', 20, 100),
                
                # 行采样比例：0.6-1.0
                'subsample': trial.suggest_float('subsample', 0.6, 1.0),
                
                # 列采样比例：0.6-1.0
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
                
                # 叶子节点最小样本数：5-50（LightGBM特有，控制叶子节点分裂）
                'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
                
                # L1正则化系数：0.001-1.0，对数尺度
                'reg_alpha': trial.suggest_float('reg_alpha', 0.001, 1.0, log=True),
                
                # L2正则化系数：0.001-1.0，对数尺度
                'reg_lambda': trial.suggest_float('reg_lambda', 0.001, 1.0, log=True),
                
                # 固定参数
                'random_state': 42,
                'verbose': -1,  # 不输出训练日志
                'n_jobs': -1  # 使用所有CPU核心
            }

            # 创建模型
            model = lgb.LGBMClassifier(**params)

            # 交叉验证
            cv = StratifiedKFold(n_splits=self.optuna_cv_folds, shuffle=True, random_state=42)
            scores = []

            for fold_idx, (train_idx, val_idx) in enumerate(cv.split(X_train, y_train)):
                X_fold_train = X_train.iloc[train_idx] if hasattr(X_train, 'iloc') else X_train[train_idx]
                y_fold_train = y_train.iloc[train_idx] if hasattr(y_train, 'iloc') else y_train[train_idx]
                X_fold_val = X_train.iloc[val_idx] if hasattr(X_train, 'iloc') else X_train[val_idx]
                y_fold_val = y_train.iloc[val_idx] if hasattr(y_train, 'iloc') else y_train[val_idx]

                # 训练模型
                model.fit(X_fold_train, y_fold_train)

                # 预测并计算准确率
                y_pred = model.predict(X_fold_val)
                fold_accuracy = accuracy_score(y_fold_val, y_pred)
                scores.append(fold_accuracy)

                # Pruning: 每个fold结束后报告中间结果
                trial.report(fold_accuracy, fold_idx)

                # 如果trial被剪枝，提前终止
                if trial.should_prune():
                    raise optuna.TrialPruned()

            # 返回平均准确率
            return np.mean(scores)

        # 创建MedianPruner
        pruner = MedianPruner(
            n_startup_trials=self.optuna_pruner_warmup,
            n_warmup_steps=self.optuna_pruner_interval
        )

        # 创建study并优化
        optuna.logging.set_verbosity(optuna.logging.WARNING)  # 减少日志输出
        study = optuna.create_study(
            direction='maximize',
            pruner=pruner,
            study_name='lightgbm_optimization'
        )

        study.optimize(
            objective,
            n_trials=self.optuna_n_trials,
            timeout=self.optuna_timeout,
            show_progress_bar=True
        )

        # 保存study和最优参数
        self.optuna_studies['LightGBM'] = study
        self.optuna_best_params['LightGBM'] = study.best_params

        # 打印结果
        print(f"\n✓ LightGBM优化完成")
        print(f"  最优准确率: {study.best_value:.4f}")
        print(f"  最优参数:")
        for key, value in study.best_params.items():
            print(f"    {key}: {value}")
        print(f"  完成试验: {len(study.trials)}")
        print(f"  剪枝试验: {len([t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED])}")

        return study.best_params

    def optimize_catboost_hyperparams(self, X_train, y_train):
        """
        使用Optuna + MedianPruner优化CatBoost超参数

        参数:
        - X_train: 训练特征
        - y_train: 训练标签

        返回:
        - best_params: 最优超参数字典
        """
        print(f"\n=== CatBoost超参数优化 ===")
        print(f"试验次数: {self.optuna_n_trials}")
        print(f"交叉验证折数: {self.optuna_cv_folds}")
        print(f"超时时间: {self.optuna_timeout}秒")

        # 定义目标函数：Optuna会多次调用此函数，每次传入不同的trial对象
        def objective(trial):
            """
            Optuna目标函数：评估一组超参数的性能
            
            参数:
            - trial: Optuna的Trial对象，用于建议超参数值和报告中间结果
            
            返回:
            - 交叉验证的平均准确率（Optuna会最大化此值）
            """
            # ========================================================================
            # CatBoost超参数搜索空间定义
            # ========================================================================
            # CatBoost参数命名与其他库略有不同：iterations（而非n_estimators）、depth（而非max_depth）
            # CatBoost对类别特征有自动处理能力，通常不需要手动编码
            params = {
                # 迭代次数：200-1000，步长100（CatBoost使用iterations而非n_estimators）
                'iterations': trial.suggest_int('iterations', 200, 1000, step=100),
                
                # 树的最大深度：4-10（CatBoost使用depth而非max_depth）
                'depth': trial.suggest_int('depth', 4, 10),
                
                # 学习率：0.01-0.3，对数尺度
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                
                # L2正则化系数：1-10（CatBoost使用l2_leaf_reg，整数而非浮点数）
                # 控制叶子节点值的L2正则化，防止过拟合
                'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 10),
                
                # 固定参数
                'random_seed': 42,  # 随机种子（CatBoost使用random_seed而非random_state）
                'verbose': False,  # 不输出训练日志
                'thread_count': -1  # 使用所有CPU核心（CatBoost使用thread_count而非n_jobs）
            }

            # 创建模型
            model = CatBoostClassifier(**params)

            # 交叉验证
            cv = StratifiedKFold(n_splits=self.optuna_cv_folds, shuffle=True, random_state=42)
            scores = []

            for fold_idx, (train_idx, val_idx) in enumerate(cv.split(X_train, y_train)):
                X_fold_train = X_train.iloc[train_idx] if hasattr(X_train, 'iloc') else X_train[train_idx]
                y_fold_train = y_train.iloc[train_idx] if hasattr(y_train, 'iloc') else y_train[train_idx]
                X_fold_val = X_train.iloc[val_idx] if hasattr(X_train, 'iloc') else X_train[val_idx]
                y_fold_val = y_train.iloc[val_idx] if hasattr(y_train, 'iloc') else y_train[val_idx]

                # 训练模型
                model.fit(X_fold_train, y_fold_train)

                # 预测并计算准确率
                y_pred = model.predict(X_fold_val)
                fold_accuracy = accuracy_score(y_fold_val, y_pred)
                scores.append(fold_accuracy)

                # Pruning: 每个fold结束后报告中间结果
                trial.report(fold_accuracy, fold_idx)

                # 如果trial被剪枝，提前终止
                if trial.should_prune():
                    raise optuna.TrialPruned()

            # 返回平均准确率
            return np.mean(scores)

        # 创建MedianPruner
        pruner = MedianPruner(
            n_startup_trials=self.optuna_pruner_warmup,
            n_warmup_steps=self.optuna_pruner_interval
        )

        # 创建study并优化
        optuna.logging.set_verbosity(optuna.logging.WARNING)  # 减少日志输出
        study = optuna.create_study(
            direction='maximize',
            pruner=pruner,
            study_name='catboost_optimization'
        )

        study.optimize(
            objective,
            n_trials=self.optuna_n_trials,
            timeout=self.optuna_timeout,
            show_progress_bar=True
        )

        # 保存study和最优参数
        self.optuna_studies['CatBoost'] = study
        self.optuna_best_params['CatBoost'] = study.best_params

        # 打印结果
        print(f"\n✓ CatBoost优化完成")
        print(f"  最优准确率: {study.best_value:.4f}")
        print(f"  最优参数:")
        for key, value in study.best_params.items():
            print(f"    {key}: {value}")
        print(f"  完成试验: {len(study.trials)}")
        print(f"  剪枝试验: {len([t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED])}")

        return study.best_params

    def save_optuna_results(self, output_dir='optuna_results'):
        """
        保存Optuna优化结果到文件

        参数:
        - output_dir: 输出目录路径（默认'optuna_results'）

        功能:
        - 保存最优参数到JSON文件（包含模型名、最优值、最优参数、试验统计等）
        - 保存优化历史到CSV文件（包含所有试验的详细记录）
        - 打印统计信息（总试验数、完成数、剪枝数、失败数等）

        输出文件:
        - {model_name}_best_params_{timestamp}.json: 最优参数JSON文件
        - {model_name}_optimization_history_{timestamp}.csv: 优化历史CSV文件
        """
        # 检查是否有优化结果可保存
        if not self.optuna_studies:
            print("⚠️ 没有Optuna优化结果可保存")
            return

        # 创建输出目录（如果不存在）
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # 生成时间戳，用于文件名
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

        print(f"\n=== 保存Optuna优化结果 ===")
        print(f"输出目录: {output_dir}")

        # 遍历每个模型的优化结果
        for model_name, study in self.optuna_studies.items():
            # ========================================================================
            # 保存最优参数到JSON文件
            # ========================================================================
            best_params_file = os.path.join(output_dir, f'{model_name}_best_params_{timestamp}.json')
            with open(best_params_file, 'w', encoding='utf-8') as f:
                json.dump({
                    'model': model_name,  # 模型名称
                    'best_value': study.best_value,  # 最优目标函数值（准确率）
                    'best_params': study.best_params,  # 最优超参数组合
                    'n_trials': len(study.trials),  # 总试验数
                    'n_pruned': len([t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]),  # 被剪枝的试验数
                    'timestamp': timestamp  # 时间戳
                }, f, indent=2, ensure_ascii=False)
            print(f"✓ {model_name} 最优参数已保存: {best_params_file}")

            # ========================================================================
            # 保存优化历史到CSV文件
            # ========================================================================
            # trials_dataframe()返回包含所有试验信息的DataFrame
            # 包括：参数值、目标函数值、试验状态、耗时等
            trials_df = study.trials_dataframe()
            history_file = os.path.join(output_dir, f'{model_name}_optimization_history_{timestamp}.csv')
            trials_df.to_csv(history_file, index=False)
            print(f"✓ {model_name} 优化历史已保存: {history_file}")

            # ========================================================================
            # 打印统计信息
            # ========================================================================
            print(f"\n{model_name} 优化统计:")
            print(f"  总试验数: {len(study.trials)}")  # 所有试验（包括完成、剪枝、失败）
            # 成功完成的试验数
            print(f"  完成试验: {len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE])}")
            # 被剪枝（提前终止）的试验数
            print(f"  剪枝试验: {len([t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED])}")
            # 失败的试验数（出现异常）
            print(f"  失败试验: {len([t for t in study.trials if t.state == optuna.trial.TrialState.FAIL])}")
            print(f"  最优准确率: {study.best_value:.4f}")  # 所有试验中的最高准确率
            print(f"  最优参数: {study.best_params}")  # 达到最优准确率的参数组合

        print(f"\n✓ 所有Optuna结果已保存到 {output_dir}")

    def train_models(self):
        """
        训练多个模型并使用Stacking集成

        模型配置：
        - RandomForest: 800棵树, max_depth=25
        - XGBoost: 800棵树, learning_rate=0.03, 带L1/L2正则化
        - LightGBM: 800棵树, num_leaves=50, 带正则化
        - CatBoost: 800迭代, depth=9
        - GradientBoosting: 500棵树, max_depth=8

        集成策略：
        - Stacking: 使用XGBoost+LightGBM+CatBoost, 元学习器为LogisticRegression
        - Voting: 使用所有5个基础模型, soft voting

        错误样本分析：
        - 自动保存最佳模型(Stacking或Voting)的预测错误样本
        - 可选：保存各基础模型的错误样本用于对比
        """
        print("\n=== 模型训练(Stacking集成) ===")

        # 准备数据
        X = self.train_combined
        y = self.train_data['player_won']

        # 使用选定的XGBoost参数对全部有标签数据进行4折交叉验证评估
        xgb_cv_params = dict(
            n_estimators=800,
            max_depth=8,
            learning_rate=0.03,
            subsample=0.85,
            colsample_bytree=0.85,
            min_child_weight=2,
            gamma=0.05,
            reg_alpha=0.1,
            reg_lambda=0.1,
            random_state=42,
            eval_metric='logloss',
            n_jobs=-1
        )
        print("\n执行XGBoost 4折交叉验证评估（全部有标签样本）...")
        xgb_cv_model = xgb.XGBClassifier(**xgb_cv_params)
        xgb_cv_splitter = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
        xgb_cv_scores = cross_val_score(
            xgb_cv_model,
            X,
            y,
            cv=xgb_cv_splitter,
            scoring='accuracy',
            n_jobs=-1
        )
        print(f"XGBoost 4折交叉验证准确率: {xgb_cv_scores.mean():.4f} ± {xgb_cv_scores.std():.4f}")

        # 数据分割 - 保存验证集索引用于后续错误分析
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=self.validation_split, random_state=42, stratify=y
        )

        # 保存验证集数据供后续分析使用
        self.X_val = X_val
        self.y_val = y_val

        # 保存验证集的原始索引
        val_indices = y_val.index.to_numpy()

        # 特征选择(可选,保留重要特征)
        # 选项1: 使用互信息特征选择
        # X_train_selected, selected_features = self.select_features(X_train, y_train, k=80)
        # X_val_selected = self.feature_selector.transform(X_val)

        # 选项2: 使用RFECV特征选择（推荐，自动选择最优特征数量）
        use_rfecv = False  # 设置为True启用RFECV特征选择
        if use_rfecv:
            X_train_selected, selected_features, optimal_n = self.select_features_rfecv(
                X_train, y_train,
                estimator=None,  # 使用默认LogisticRegression，也可以传入其他估计器如RandomForestClassifier(n_estimators=100)
                cv=4,  # 5折交叉验证
                scoring='accuracy',  # 评分标准
                min_features_to_select=20,  # 最少保留20个特征
                n_jobs=-1  # 使用所有CPU核心
            )
            X_val_selected = self.feature_selector.transform(X_val)
            # 使用选择后的特征
            X_train = X_train_selected
            X_val = X_val_selected
            print(f"\n使用RFECV选择后的特征进行模型训练")
            print(f"训练特征形状: {X_train.shape}, 验证特征形状: {X_val.shape}")

        # ========================================================================
        # Optuna超参数优化流程（可选）
        # ========================================================================
        # 如果启用Optuna优化，会在训练模型之前先进行超参数搜索
        # 优化过程可能耗时较长（每个模型可能需要数小时），但通常能提升模型性能
        # ========================================================================
        xgb_best_params = None  # XGBoost最优参数（如果进行了优化）
        lgb_best_params = None   # LightGBM最优参数（如果进行了优化）
        cat_best_params = None  # CatBoost最优参数（如果进行了优化）

        if self.use_optuna_tuning:
            print(f"\n{'='*80}")
            print("=== 开始Optuna超参数优化 ===")
            print(f"{'='*80}")
            print(f"优化配置:")
            print(f"  - 每个模型试验次数: {self.optuna_n_trials}")  # 默认50次
            print(f"  - 交叉验证折数: {self.optuna_cv_folds}")  # 默认3折
            print(f"  - 单个模型超时: {self.optuna_timeout}秒")  # 默认3600秒（1小时）
            print(f"  - Pruner预热步数: {self.optuna_pruner_warmup}")  # 默认前5个试验不剪枝

            # 优化XGBoost超参数
            # 使用TPE算法和MedianPruner进行智能搜索，返回最优参数组合
            xgb_best_params = self.optimize_xgboost_hyperparams(X_train, y_train)

            # 优化LightGBM超参数
            # 搜索空间包括num_leaves、min_child_samples等LightGBM特有参数
            lgb_best_params = self.optimize_lightgbm_hyperparams(X_train, y_train)

            # 优化CatBoost超参数
            # 搜索空间包括iterations、depth、l2_leaf_reg等CatBoost参数
            cat_best_params = self.optimize_catboost_hyperparams(X_train, y_train)

            # 保存Optuna优化结果到文件
            # 包括最优参数JSON文件和优化历史CSV文件
            self.save_optuna_results(output_dir='optuna_results')

            print(f"\n{'='*80}")
            print("=== Optuna超参数优化完成 ===")
            print(f"{'='*80}")
        else:
            print("\n⚠️ Optuna超参数优化已禁用，使用默认参数")

        # ========================================================================
        # 准备最终训练数据
        # ========================================================================
        X_train_final = X_train.copy()
        y_train_final = y_train.copy()

        # ========================================================================
        # 正式模型训练（使用最终确定的训练数据）
        # ========================================================================
        print(f"\n=== 正式模型训练 ===")
        print(f"训练数据大小: {len(X_train_final)} 样本")

        # ========================================================================
        # 定义基础模型参数（根据是否使用Optuna优化选择参数）
        # ========================================================================
        # 如果启用了Optuna优化且优化成功，使用优化后的最优参数
        # 否则使用经验调优的默认参数
        # ========================================================================
        
        # XGBoost参数选择
        if self.use_optuna_tuning and xgb_best_params:
            # 使用Optuna优化的XGBoost参数
            # 合并优化参数和固定参数（确保random_state等固定参数不被覆盖）
            print("\n✓ 使用Optuna优化的XGBoost参数")
            xgb_params = {
                **xgb_best_params,  # 展开优化后的参数
                'random_state': 42,  # 固定随机种子
                'eval_metric': 'logloss',  # 固定评估指标
                'n_jobs': -1  # 使用所有CPU核心
            }
        else:
            # 使用默认XGBoost参数（经验调优的参数）
            xgb_params = xgb_cv_params

        # LightGBM参数选择
        if self.use_optuna_tuning and lgb_best_params:
            # 使用Optuna优化的LightGBM参数
            print("✓ 使用Optuna优化的LightGBM参数")
            lgb_params = {
                **lgb_best_params,  # 展开优化后的参数
                'random_state': 42,  # 固定随机种子
                'verbose': -1,  # 不输出训练日志
                'n_jobs': -1  # 使用所有CPU核心
            }
        else:
            # 使用默认LightGBM参数（经验调优的参数）
            lgb_params = {
                'n_estimators': 800,  # 树的数量
                'max_depth': 10,  # 树的最大深度
                'learning_rate': 0.03,  # 学习率
                'subsample': 0.85,  # 行采样比例
                'colsample_bytree': 0.85,  # 列采样比例
                'min_child_samples': 15,  # 叶子节点最小样本数
                'reg_alpha': 0.1,  # L1正则化系数
                'reg_lambda': 0.1,  # L2正则化系数
                'num_leaves': 50,  # 叶子节点数
                'random_state': 42,
                'verbose': -1,
                'n_jobs': -1
            }

        # CatBoost参数选择
        if self.use_optuna_tuning and cat_best_params:
            # 使用Optuna优化的CatBoost参数
            print("✓ 使用Optuna优化的CatBoost参数")
            cat_params = {
                **cat_best_params,  # 展开优化后的参数
                'random_seed': 42,  # 固定随机种子（CatBoost使用random_seed）
                'verbose': False,  # 不输出训练日志
                'thread_count': -1  # 使用所有CPU核心（CatBoost使用thread_count）
            }
        else:
            # 使用默认CatBoost参数（经验调优的参数）
            cat_params = {
                'iterations': 800,  # 迭代次数（CatBoost使用iterations）
                'depth': 9,  # 树的最大深度（CatBoost使用depth）
                'learning_rate': 0.03,  # 学习率
                'l2_leaf_reg': 2,  # L2正则化系数
                'random_seed': 42,
                'verbose': False,
                'thread_count': -1
            }

        base_models = {
            'XGBoost': xgb.XGBClassifier(**xgb_params),
            'LightGBM': lgb.LGBMClassifier(**lgb_params),
            'CatBoost': CatBoostClassifier(**cat_params)
        }

        # 训练基础模型（使用最终的训练数据）
        for name, model in base_models.items():
            print(f"训练 {name}...")
            model.fit(X_train_final, y_train_final)

            y_pred = model.predict(X_val)
            accuracy = accuracy_score(y_val, y_pred)

            print(f"{name} 验证准确率: {accuracy:.4f}")

            self.models[name] = model

            if hasattr(model, 'feature_importances_'):
                self.feature_importance[name] = model.feature_importances_

        # Stacking集成模型 - 使用较少的基础模型减少内存占用
        print("\n训练Stacking集成模型...")

        # 选择表现最好的3个模型进行Stacking
        stacking_estimators = [
            ('XGBoost', base_models['XGBoost']),
            ('LightGBM', base_models['LightGBM']),
            ('CatBoost', base_models['CatBoost'])
        ]

        # 方案2: 使用'prefit'避免交叉验证（最快，但可能略微降低性能）
        # 注意：prefit需要基础模型已经训练好，这里我们已经训练过了
        stacking_model = StackingClassifier(
            estimators=stacking_estimators,
            final_estimator=LogisticRegression(C=1.0, max_iter=500, random_state=42),  # 减少迭代次数
            cv=2, # cv=4, # cv='prefit',  # 使用prefit避免交叉验证，速度提升3-5倍！ # 表示在训练Stacking终结器时使用3折交叉验证（内部重训练3次取平均） StackingClassifier 在 cv=3 时会把训练集再按 3 折划分，用前两折生成基础模型的预测、第三折训练终结器；然后轮换 3 次，每次换一折做"终结器训练集"，最后把三次训练得到的终结器权重平均再用于最终模型。 先用那 2 折把所有基础模型（XGBoost、LightGBM、CatBoost 等）重新训练好，然后让它们去预测刚刚被留出来的那 1 折，得到这一折在各个基础模型下的预测结果（这些预测值就像新的特征）。使用这份折上的基础预测结果 + 实际标签，去训练终结器（这里是 LogisticRegression）。
            n_jobs=1  # 保持单线程
        )

        try:
            stacking_model.fit(X_train_final, y_train_final)
            y_pred_stacking = stacking_model.predict(X_val)
            stacking_accuracy = accuracy_score(y_val, y_pred_stacking)
            print(f"Stacking集成模型验证准确率: {stacking_accuracy:.4f}")
            self.models['Stacking'] = stacking_model
        except Exception as e:
            print(f"Stacking训练失败: {e}")
            print("跳过Stacking模型,使用Voting作为主要集成方法")

        # # Voting集成(soft voting)
        # print("\n训练Voting集成模型...")
        # voting_estimators = list(base_models.items())
        # voting_model = VotingClassifier(
        #     estimators=voting_estimators,
        #     voting='soft',
        #     n_jobs=-1
        # )
        # voting_model.fit(X_train_final, y_train_final)
        # y_pred_voting = voting_model.predict(X_val)
        # voting_accuracy = accuracy_score(y_val, y_pred_voting)
        # print(f"Voting集成模型验证准确率: {voting_accuracy:.4f}")

        # self.models['Voting'] = voting_model

        # 保存最佳模型的预测错误样本
        if 'Stacking' in self.models:
            self.best_model_name = 'Stacking'
            self.best_model = self.models['Stacking']
            best_predictions = self.models['Stacking'].predict(X_val)
        else:
            self.best_model_name = 'Voting'
            self.best_model = self.models['Voting']
            best_predictions = y_pred_voting

        # 保存错误样本到JSON
        # self.save_misclassified_samples(val_indices, best_predictions, y_val, self.best_model_name)

        # # 也可以保存各个基础模型的错误样本用于对比分析
        # print("\n=== 保存各基础模型的错误样本 ===")
        # for name, model in base_models.items():
        #     y_pred_base = model.predict(X_val)
        #     self.save_misclassified_samples(val_indices, y_pred_base, y_val, name)

        return self.models

    def make_predictions(self):
        """生成预测结果"""
        print("\n=== 生成预测 ===")

        # 如果使用了特征选择，需要对测试数据也进行转换
        test_features = self.test_combined
        if hasattr(self, 'feature_selector') and self.feature_selector is not None:
            print("应用特征选择到测试数据...")
            test_features_array = self.feature_selector.transform(self.test_combined)
            # 如果原始数据是DataFrame，将结果也转换为DataFrame
            if isinstance(self.test_combined, pd.DataFrame):
                selected_features = self.test_combined.columns[self.feature_selector.support_].tolist()
                test_features = pd.DataFrame(
                    test_features_array,
                    columns=selected_features,
                    index=self.test_combined.index
                )
            else:
                test_features = test_features_array
            print(f"测试特征形状: {test_features.shape}")

        predictions = {}

        for name, model in self.models.items():
            pred = model.predict(test_features)
            predictions[name] = pred
            print(f"{name} 预测完成")

        # 优先选择Stacking,如果没有则用Voting
        if 'Stacking' in predictions:
            best_predictions = predictions['Stacking']
            print("使用Stacking模型生成最终预测")
        else:
            best_predictions = predictions['Voting']
            print("使用Voting模型生成最终预测")

        # 创建提交文件
        submission = pd.DataFrame({
            'battle_id': self.test_data['battle_id'],
            'player_won': best_predictions.astype(int)
        })

        submission.to_csv('submission_enhanced_v4.csv', index=False)
        print("提交文件已生成: submission_enhanced_v4.csv")

        return submission, predictions

    def get_sample_features(self, original_index):
        """获取指定索引样本的所有特征"""
        if self.train_combined is None:
            print("错误: 请先运行特征提取 (extract_static_features, extract_dynamic_features, combine_features)")
            return None

        if original_index >= len(self.train_combined):
            print(f"错误: 索引超出范围 (最大: {len(self.train_combined)-1})")
            return None

        return self.train_combined.iloc[original_index].to_dict()

    def explain_single_prediction(self, sample_index, model_name='XGBoost', top_k=20, use_shap=True):
        """
        解释单个样本的预测结果
        
        功能：
        1. 显示该样本的所有特征值
        2. 显示预测结果和概率
        3. 显示对该预测贡献最大的特征（使用SHAP或特征重要性）
        
        参数：
        - sample_index: 样本索引（在train_combined中的索引）
        - model_name: 使用的模型名称（默认'XGBoost'）
        - top_k: 显示前K个最重要的特征（默认20）
        - use_shap: 是否使用SHAP值（如果可用，默认True）
        
        返回：
        - 包含预测解释的字典
        """
        print("\n" + "="*80)
        print(f"解释样本 #{sample_index} 的预测结果")
        print("="*80)
        
        # 检查模型和数据
        if self.train_combined is None:
            print("错误: 请先运行特征提取")
            return None
        
        if model_name not in self.models:
            print(f"错误: 模型 '{model_name}' 不存在")
            print(f"可用模型: {list(self.models.keys())}")
            return None
        
        model = self.models[model_name]
        
        # 获取样本特征
        if sample_index >= len(self.train_combined):
            print(f"错误: 索引超出范围 (最大: {len(self.train_combined)-1})")
            return None
        
        sample_features = self.train_combined.iloc[sample_index:sample_index+1]
        feature_names = self.train_combined.columns.tolist()
        
        # 获取预测结果
        prediction = model.predict(sample_features)[0]
        prediction_proba = model.predict_proba(sample_features)[0]
        
        print(f"\n📊 预测结果:")
        print(f"   预测类别: {prediction} ({'P1获胜' if prediction == 1 else 'P2获胜'})")
        print(f"   预测概率: P1获胜={prediction_proba[1]:.4f}, P2获胜={prediction_proba[0]:.4f}")
        print(f"   置信度: {max(prediction_proba):.4f}")
        
        # 尝试使用SHAP值解释
        if use_shap:
            try:
                import shap
                print(f"\n🔍 使用SHAP值解释预测（显示前{top_k}个特征）...")
                
                # 创建SHAP解释器
                explainer = shap.TreeExplainer(model)
                
                # ============================================
                # 背景数据（Background Data）说明
                # ============================================
                # SHAP值计算的是"相对于某个基准的贡献"
                # 
                # 【背景数据的作用】
                # 1. 作为基准参考点：SHAP值衡量的是"相对于背景数据的平均情况，该特征值如何影响预测"
                # 2. 提高计算效率：使用少量代表性样本（如100个）作为背景，而不是整个数据集
                # 3. 更稳定的解释：使用多个样本的平均值作为基准，比单个样本更稳定
                #
                # 【示例理解】
                # 假设要解释样本A的预测：
                # - 背景数据：100个验证集样本的平均特征值
                # - SHAP值计算：样本A的特征值 vs 背景数据的平均特征值
                # - 结果：每个特征对预测的贡献 = 该特征在样本A中的值相对于背景平均值的差异带来的影响
                #
                # 【为什么使用验证集作为背景】
                # - 验证集代表模型的"典型输入分布"
                # - 比训练集更接近真实预测场景
                # - 避免使用测试数据（可能泄露信息）
                #
                # 【计算方式对比】
                # - 有背景数据：SHAP值 = f(样本特征) - f(背景平均特征)  [更准确，需要背景]
                # - 无背景数据：SHAP值 = f(样本特征) - f(模型期望值)   [快速，但可能不够准确]
                # ============================================
                
                if self.X_val is not None and len(self.X_val) > 100:
                    # 使用验证集的样本作为背景数据（取前100个代表性样本）
                    # 这100个样本的平均特征值将作为SHAP值计算的基准参考点
                    background_data = self.X_val.iloc[:100]
                    shap_values = explainer.shap_values(sample_features, background_data)
                else:
                    # 如果没有验证集或验证集太小，不使用背景数据
                    # SHAP将使用模型内部的期望值作为基准（计算更快，但解释可能不如有背景数据准确）
                    shap_values = explainer.shap_values(sample_features)
                
                # 处理SHAP值的格式（可能是列表或数组）
                if isinstance(shap_values, list):
                    shap_values = shap_values[1] if len(shap_values) > 1 else shap_values[0]
                
                # 获取该样本的SHAP值
                if len(shap_values.shape) > 1:
                    sample_shap = shap_values[0]
                else:
                    sample_shap = shap_values
                
                # 创建特征贡献DataFrame
                feature_contributions = pd.DataFrame({
                    'feature': feature_names,
                    'shap_value': sample_shap,
                    'feature_value': sample_features.iloc[0].values,
                    'abs_shap': np.abs(sample_shap)
                })
                
                # 按绝对SHAP值排序
                feature_contributions = feature_contributions.sort_values('abs_shap', ascending=False)
                
                print(f"\n📈 对该预测贡献最大的前{top_k}个特征:")
                print("-" * 80)
                print(f"{'排名':<6} {'特征名':<40} {'特征值':<15} {'SHAP值':<12} {'贡献方向':<10}")
                print("-" * 80)
                
                for i, (idx, row) in enumerate(feature_contributions.head(top_k).iterrows(), 1):
                    direction = "↑ 支持P1" if row['shap_value'] > 0 else "↓ 支持P2"
                    print(f"{i:<6} {row['feature']:<40} {row['feature_value']:<15.4f} {row['shap_value']:<12.6f} {direction:<10}")
                
                # 计算总贡献
                positive_contrib = feature_contributions[feature_contributions['shap_value'] > 0]['shap_value'].sum()
                negative_contrib = feature_contributions[feature_contributions['shap_value'] < 0]['shap_value'].sum()
                
                print(f"\n💡 贡献总结:")
                print(f"   支持P1获胜的总贡献: {positive_contrib:.4f}")
                print(f"   支持P2获胜的总贡献: {abs(negative_contrib):.4f}")
                print(f"   净贡献: {positive_contrib + negative_contrib:.4f}")
                
                return {
                    'sample_index': sample_index,
                    'prediction': int(prediction),
                    'prediction_proba': prediction_proba,
                    'feature_contributions': feature_contributions,
                    'method': 'SHAP'
                }
                
            except ImportError:
                print("⚠️ SHAP库未安装，使用特征重要性方法...")
                use_shap = False
            except Exception as e:
                print(f"⚠️ SHAP计算失败: {e}")
                print("使用特征重要性方法...")
                use_shap = False
        
        # # 备选方案：使用特征重要性
        # if not use_shap:
        #     print(f"\n🔍 使用特征重要性解释预测（显示前{top_k}个特征）...")
            
        #     # 获取特征重要性
        #     if hasattr(model, 'feature_importances_'):
        #         importances = model.feature_importances_
        #     else:
        #         print("错误: 模型没有feature_importances_属性")
        #         return None
            
        #     # 获取样本的特征值
        #     sample_values = sample_features.iloc[0].values
            
        #     # 创建特征贡献DataFrame
        #     feature_contributions = pd.DataFrame({
        #         'feature': feature_names,
        #         'importance': importances,
        #         'feature_value': sample_values,
        #         'weighted_contribution': importances * np.abs(sample_values)
        #     })
            
        #     # 按加权贡献排序
        #     feature_contributions = feature_contributions.sort_values('weighted_contribution', ascending=False)
            
        #     print(f"\n📈 对该预测贡献最大的前{top_k}个特征:")
        #     print("-" * 80)
        #     print(f"{'排名':<6} {'特征名':<40} {'特征值':<15} {'重要性':<12} {'加权贡献':<12}")
        #     print("-" * 80)
            
        #     for i, (idx, row) in enumerate(feature_contributions.head(top_k).iterrows(), 1):
        #         print(f"{i:<6} {row['feature']:<40} {row['feature_value']:<15.4f} {row['importance']:<12.6f} {row['weighted_contribution']:<12.6f}")
            
        #     return {
        #         'sample_index': sample_index,
        #         'prediction': int(prediction),
        #         'prediction_proba': prediction_proba,
        #         'feature_contributions': feature_contributions,
        #         'method': 'Feature Importance'
        #     }
        
        return None

    def analyze_results(self):
        """分析结果"""
        print("\n=== 结果分析 ===")

        # 特征重要性分析
        if self.feature_importance:
            print("\n特征重要性分析:")
            for model_name, importance in self.feature_importance.items():
                print(f"\n{model_name} 前15个重要特征:")
                feature_names = self.train_combined.columns
                importance_df = pd.DataFrame({
                    'feature': feature_names,
                    'importance': importance
                }).sort_values('importance', ascending=False)
                print(importance_df.head(15))

        return self.feature_importance

def main():
    """主函数"""
    # 初始化日志系统
    log_file_path = setup_logging(log_dir='print_log')
    print(f"日志系统已启动，输出将保存到: {log_file_path}")
    
    try:
        print("Pokemon Battles Prediction 2025 - Enhanced Version")
        print("=" * 60)

        # 创建预测器实例
        predictor = PokemonBattlePredictorEnhanced()
        # 加载数据
        if os.path.exists('input/train.jsonl') and os.path.exists('input/test.jsonl'):
            train_data, test_data = predictor.load_data('input/train.jsonl', 'input/test.jsonl')
        else:
            train_data, test_data = predictor.load_data('../input/fds-pokemon-battles-prediction-2025/train.jsonl', '../input/fds-pokemon-battles-prediction-2025/test.jsonl')

        # 特征工程
        predictor.extract_static_features()
        predictor.extract_dynamic_features()
        predictor.combine_features()

        # 模型训练
        predictor.train_models()

        # 生成预测
        submission, predictions = predictor.make_predictions()

        # 结果分析
        predictor.analyze_results()
        
        # 特征移除影响测试
        print("\n" + "="*60)
        print("开始特征移除影响测试...")
        print("="*60)
        # removal_results = predictor.test_feature_removal_impact(top_n=20, n_repeats=2)
        

        # 示例：解释单个样本的预测（索引6678）
        # print("\n" + "="*60)
        # print("解释样本 #6678 的预测结果...")
        # print("="*60)
        explanation = predictor.explain_single_prediction(
            sample_index= 3907,#7433, # 238  6678, # 8440, #6678    11这个确实是特例     7433存活但是血量低
            model_name='XGBoost',
            top_k=40,
            use_shap=True  # 如果安装了SHAP库，会使用SHAP值；否则使用特征重要性
        )

        print("\n任务完成!")
        print("请检查 submission_enhanced_v4.csv 文件")
        print("特征分析报告已生成")
        print("\n💡 提示: 要解释单个样本的预测，可以使用:")
        print("   predictor.explain_single_prediction(sample_index=6678, model_name='XGBoost', top_k=20)")

    except FileNotFoundError as e:
        print(f"错误: 找不到数据文件 - {e}")
        print("请确保 train.jsonl 和 test.jsonl 文件存在于 input/ 目录")
    except Exception as e:
        print(f"发生错误: {e}")
        import traceback
        print(traceback.format_exc())
    finally:
        # 关闭日志文件
        close_logging()
        print(f"\n日志文件已保存: {log_file_path}")

if __name__ == "__main__":
    main()


日志系统已启动，输出将保存到: print_log/print_log_20251114_224113.log
Pokemon Battles Prediction 2025 - Enhanced Version
正在加载数据...
训练数据加载完成: 10000 条记录
测试数据加载完成: 5000 条记录

=== 提取静态特征(增强版) ===
静态特征提取完成: 34 个特征

=== 提取动态特征(增强版) ===
动态特征提取完成: 30 个特征

=== 合并特征并创建交互项 ===
特征合并完成:
训练特征形状: (10000, 65)
测试特征形状: (5000, 65)

=== 模型训练(Stacking集成) ===

执行XGBoost 4折交叉验证评估（全部有标签样本）...
XGBoost 4折交叉验证准确率: 0.8234 ± 0.0034

⚠️ Optuna超参数优化已禁用，使用默认参数

=== 正式模型训练 ===
训练数据大小: 9000 样本
训练 XGBoost...
XGBoost 验证准确率: 0.8190
训练 LightGBM...
LightGBM 验证准确率: 0.8300
训练 CatBoost...
CatBoost 验证准确率: 0.8230

训练Stacking集成模型...
Stacking集成模型验证准确率: 0.8250

=== 生成预测 ===
XGBoost 预测完成
LightGBM 预测完成
CatBoost 预测完成
Stacking 预测完成
使用Stacking模型生成最终预测
提交文件已生成: submission_enhanced_v4.csv

=== 结果分析 ===

特征重要性分析:

XGBoost 前15个重要特征:
                            feature  importance
59                   fnt_count_diff    0.170987
58                  fnt_count_ratio    0.118259
54  p2_unique_pokemon_count_30turns    0.105281
40      abnormal_status_count_rati